In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth
pd.options.mode.chained_assignment = None  # default='warn'

data_2021 = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv', low_memory = False, encoding='UTF-8')
questions_2021 = data_2021.iloc[0, :].T
data_2021 = data_2021.iloc[1:, :]
data_2020 = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv', low_memory = False, encoding='UTF-8')
questions_2020 = data_2020.iloc[0, :].T
data_2020 = data_2020.iloc[1:, :]
data_2019 = pd.read_csv('../input/kaggle-survey-2019/multiple_choice_responses.csv', low_memory = False, encoding='UTF-8')
questions_2019 = data_2019.iloc[0, :].T
data_2019 = data_2019.iloc[1:, :]
data_2018 = pd.read_csv('../input/kaggle-survey-2018/multipleChoiceResponses.csv', low_memory = False, encoding='UTF-8')
questions_2018 = data_2018.iloc[0, :].T
data_2018 = data_2018.iloc[1:, :]
data_2017 = pd.read_csv('../input/kaggle-survey-2017/multipleChoiceResponses.csv', low_memory = False, encoding='ISO-8859-1')
questions_2017 = data_2017.iloc[0, :].T
data_2017 = data_2017.iloc[1:, :]

!pip install openpyxl

nvda_earnings = pd.read_excel('../input/stonks-data/nvda_earnings.xlsx')
cloud_earnings = pd.read_excel('../input/stonks-data/cloud.xlsx')
# 2021 Yes. 2020 Yes. 2019 Yes. About TPUs

def get_professionals(data, column):
    data = data.loc[data[column] != 'Student']
    data = data.loc[data[column] != 'Currently not employed']
    data = data.loc[data[column] != 'Not employed']
    data = data.loc[data[column].notna()]
    return data

pros_2021 = get_professionals(data_2021, 'Q5')
pros_2020 = get_professionals(data_2020, 'Q5')

pros_2019 = data_2019[data_2019['Q5'].notna()]
pros_2019 = pros_2019[pros_2019['Q5'] != 'Student']
pros_2019 = pros_2019[pros_2019['Q5'] != 'Not employed']

In [ ]:
def open_source(x):
    if x == 'MySQL':
        return 1
    elif x == 'PostgreSQL':
        return 1
    elif x == 'MongoDB':
        return 1
    elif x == 'SQLite':
        return 1
    elif x == 'PostgresSQL':
        return 1
    else:
        return 0
    
def aws(x):
    if 'Amazon' in x:
        return 1
    elif 'AWS' in x:
        return 1
    else:
        return 0
    
def gcp(x):
    if 'Google' in x:
        return 1
    else:
        return 0
    
def azure(x):
    if 'Microsoft' in x:
        return 1
    elif 'Azure' in x:
        return 1
    else: 
        return 0
    
def annotate_axes(fig):
    for i, ax in enumerate(fig.axes):
        ax.text(0.5, 0.5, "ax%d" % (i+1), va="center", ha="center")
        ax.tick_params(labelbottom=False, labelleft=False)

In [ ]:
# --- 2021 Q.27 Cloud Computing Platform ---

cloud_2021 = ['Q27_A_Part_1','Q27_A_Part_2','Q27_A_Part_3','Q27_A_Part_4','Q27_A_Part_5','Q27_A_Part_6','Q27_A_Part_7','Q27_A_Part_8', 'Q27_A_Part_9','Q27_A_Part_10','Q27_A_Part_11','Q27_A_OTHER']
df_2021 = pros_2021[cloud_2021]

count_2021 = pd.Series(df_2021[cloud_2021].squeeze().values.ravel()).value_counts()

df_count_2021 = pd.DataFrame(count_2021)
df_count_2021 = df_count_2021.reset_index()
df_count_2021.columns = ['Cloud', 'Counts']

# --- 2020 Q.26 Cloud Computing Platform ---

cloud_2020 = ['Q26_A_Part_1','Q26_A_Part_2','Q26_A_Part_3','Q26_A_Part_4','Q26_A_Part_5','Q26_A_Part_6', 'Q26_A_Part_7','Q26_A_Part_8','Q26_A_Part_9','Q26_A_Part_10','Q26_A_Part_11','Q26_A_OTHER']
df_2020 = pros_2020[cloud_2020]

count_2020 = pd.Series(df_2020[cloud_2020].squeeze().values.ravel()).value_counts()

df_count_2020 = pd.DataFrame(count_2020)
df_count_2020 = df_count_2020.reset_index()
df_count_2020.columns = ['Cloud', 'Counts']

# --- 2019 Q.29 Cloud Computing Platform ---

cloud_2019 = ['Q29_Part_1','Q29_Part_2','Q29_Part_3','Q29_Part_4','Q29_Part_5','Q29_Part_6','Q29_Part_7', 'Q29_Part_8','Q29_Part_9','Q29_Part_10','Q29_Part_11','Q29_Part_12']
df_2019 = pros_2019[cloud_2019]

count_2019 = pd.Series(df_2019[cloud_2019].squeeze().values.ravel()).value_counts()

df_count_2019 = pd.DataFrame(count_2019)
df_count_2019 = df_count_2019.reset_index()
df_count_2019.columns = ['Cloud', 'Counts']
df_count_2019 = df_count_2019.append({'Cloud':'IBM Cloud / Red Hat', 'Counts': 451}, ignore_index=True).append({'Cloud':'Tencent Cloud', 'Counts': None}, ignore_index=True)
df_count_2019 = df_count_2019.drop([4,11]).reset_index(drop = True)

# --- Merge All Three ---

cloud_df = df_count_2021.merge(df_count_2020, on = 'Cloud').merge(df_count_2019, how = 'left')
cloud_df['Counts'][4] = 451
cloud_df = cloud_df.rename(columns = {'Cloud': 'Cloud', 'Counts_x':'2021', 'Counts_y': '2020', 'Counts': '2019'})
cloud_df_bar = cloud_df.set_index('Cloud').T[::-1]

cloud_df_bar.columns = cloud_df_bar.columns.str.strip()
cloud_df_bar = cloud_df_bar.rename(columns = {'Amazon Web Services (AWS)': 'AWS',
                                             'Google Cloud Platform (GCP)': 'GCP',
                                             'Microsoft Azure': 'Azure',
                                             'IBM Cloud / Red Hat': 'IBM',
                                             'Oracle Cloud': 'Oracle',
                                              'VMware Cloud': 'VMware',
                                              'SAP Cloud': 'SAP',
                                              'Salesforce Cloud': 'Salesforce',
                                              'Alibaba Cloud': 'Alibaba',
                                              'Tencent Cloud': 'Tencent'
                                             })

# --- Plot ---

plt.style.use('classic')
plt.rcParams['font.size'] = 20

fig, ax = plt.subplots(figsize = (10,10))

cloud_df_bar.plot(kind = 'bar', 
                  ax = ax,
                  color = ('#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f','#7f7f7f'),
                  width = 1)

ax.set_title("Cloud computing platform usage", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.8), fontsize=20)

plt.savefig('1.1 Cloud Computing Platform Usage.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Cloud Computing Platform Market Share ---

for_perc = cloud_df_bar
for_perc = for_perc.divide(for_perc.sum(axis=1), axis = 0)

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))
for_perc.plot(kind='area',
              stacked=True,
              ax = ax,
              color = ('#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f','#7f7f7f')
             )

ax.annotate('Share of None is decreasing',
             xy=(1.6, 0.53),
             xytext=(0.3, 0.58),
             arrowprops = dict(facecolor='black', shrink=0.05),
             fontsize=20,
            )

ax.set_title("Cloud computing platform usage by share", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Percentage', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.8), fontsize=20)

plt.savefig('1.2 Cloud Computing Platform Usage by share.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- 2021 Q27. The best developer's experience with ---

cloud_2_year_list = ['Q27_A_Part_1',
          'Q27_A_Part_2',
          'Q27_A_Part_3',
          'Q27_A_Part_4',
          'Q27_A_Part_5',
          'Q27_A_Part_6',
          'Q27_A_Part_7',
          'Q27_A_Part_8',
          'Q27_A_Part_9',
          'Q27_A_Part_10',
          'Q27_A_Part_11',
          'Q27_A_OTHER']

cloud_2_year_later = pros_2021[cloud_2_year_list]

count_cloud_2_year_later = pd.Series(cloud_2_year_later[cloud_2_year_list].squeeze().values.ravel()).value_counts()
df_count_cloud_2_year_later = pd.DataFrame(count_cloud_2_year_later).reset_index()
df_count_cloud_2_year_later.columns = ['Cloud', '2021']
df_count_cloud_2_year_later = df_count_cloud_2_year_later.set_index('Cloud').T


cloud_best_exp = pd.Series(pros_2021['Q28'].squeeze().values.ravel()).value_counts()
df_cloud_best_exp = pd.DataFrame(cloud_best_exp).reset_index()
df_cloud_best_exp.columns = ['Cloud', '2021']
df_cloud_best_exp = df_cloud_best_exp.set_index('Cloud').T
df_cloud_best_exp.columns = df_cloud_best_exp.columns.str.strip()
df_cloud_best_exp = df_cloud_best_exp.rename(columns = {'Amazon Web Services (AWS)': 'AWS',
                                             'Google Cloud Platform (GCP)': 'GCP',
                                            'They all had a similarly enjoyable developer experience':'All similar',
                                             'Microsoft Azure': 'Azure',
                                                        'None were satisfactory':'All bad',
                                             'IBM Cloud / Red Hat': 'IBM',
                                             'Oracle Cloud': 'Oracle',
                                              'VMware Cloud': 'VMware',
                                              'SAP Cloud': 'SAP',
                                              'Salesforce Cloud': 'Salesforce',
                                              'Alibaba Cloud': 'Alibaba',
                                              'Tencent Cloud': 'Tencent'
                                             })

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))

df_cloud_best_exp.plot(kind = 'bar',
                       ax = ax,
                       color = ('#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f','#7f7f7f', '#7f7f7f'))
                        # AWS = Blue, GCP = Red, Azure = Orange

ax.set_title("Cloud platform with the best developer experience", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Percentage', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.8), fontsize=20)

plt.savefig('1.3 Cloud Platform with the best developer experience.png', bbox_inches='tight')

plt.show()

In [ ]:
# --- 2021 Q27_B Future Preference ---

cloud_2_year_list = ['Q27_B_Part_1',
          'Q27_B_Part_2',
          'Q27_B_Part_3',
          'Q27_B_Part_4',
          'Q27_B_Part_5',
          'Q27_B_Part_6',
          'Q27_B_Part_7',
          'Q27_B_Part_8',
          'Q27_B_Part_9',
          'Q27_B_Part_10',
          'Q27_B_Part_11',
          'Q27_B_OTHER']

cloud_2_year_later = pros_2021[cloud_2_year_list]

count_cloud_2_year_later = pd.Series(cloud_2_year_later[cloud_2_year_list].squeeze().values.ravel()).value_counts()
df_count_cloud_2_year_later = pd.DataFrame(count_cloud_2_year_later).reset_index()
df_count_cloud_2_year_later.columns = ['Cloud', '2021']
df_count_cloud_2_year_later = df_count_cloud_2_year_later.set_index('Cloud').T

df_count_cloud_2_year_later.columns = df_count_cloud_2_year_later.columns.str.strip()
df_count_cloud_2_year_later = df_count_cloud_2_year_later.rename(columns = {'Amazon Web Services (AWS)': 'AWS',
                                             'Google Cloud Platform (GCP)': 'GCP',
                                             'Microsoft Azure': 'Azure',
                                             'IBM Cloud / Red Hat': 'IBM',
                                             'Oracle Cloud': 'Oracle',
                                              'VMware Cloud': 'VMware',
                                              'SAP Cloud': 'SAP',
                                              'Salesforce Cloud': 'Salesforce',
                                              'Alibaba Cloud': 'Alibaba',
                                              'Tencent Cloud': 'Tencent'
                                             })


fig, ax = plt.subplots(figsize = (10,10))

df_count_cloud_2_year_later.plot(kind='bar',
                                 ax=ax,
                                color = ('#1f77b4', '#ff7f0e', '#d62728', '#2ca02c', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f','#7f7f7f', '#7f7f7f'))
                                # Azure = Red = #d62728 // None = Green = #2ca02c
ax.set_title("Cloud platform that respondant is willing to become more familiar in 2 year", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.8), fontsize=20)

plt.savefig('1.4 Cloud platform that respondant is willing to become more familiar in 2 year.png', bbox_inches='tight')

plt.show()

In [ ]:
# --- Cloud Computing PRODUCT ---

# --- 2019 Q30 ---

cloud_computing_2019 = ['Q30_Part_1','Q30_Part_2','Q30_Part_3','Q30_Part_4','Q30_Part_5','Q30_Part_6','Q30_Part_7','Q30_Part_8','Q30_Part_9','Q30_Part_10','Q30_Part_11','Q30_OTHER_TEXT']
df_computing_2019 = pros_2019[cloud_computing_2019]
count_compute_2019 = pd.Series(df_computing_2019[cloud_computing_2019].squeeze().values.ravel()).value_counts()

# ---
df_count_compute_2019 = pd.DataFrame(count_compute_2019)
df_count_compute_2019 = df_count_compute_2019.reset_index()
df_count_compute_2019.columns = ['Cloud Compute', 'Counts']

# ---
df_count_compute_2019['Cloud Compute'] = df_count_compute_2019['Cloud Compute'].str.strip()
df_count_compute_2019 = df_count_compute_2019[(df_count_compute_2019['Cloud Compute'] == 'AWS Elastic Compute Cloud (EC2)') |
                      (df_count_compute_2019['Cloud Compute'] == 'Google Compute Engine (GCE)') |
                     (df_count_compute_2019['Cloud Compute'] == 'Azure Virtual Machines') |
                     (df_count_compute_2019['Cloud Compute'] == 'None')
                                             ]
# ---
df_count_compute_2019['Cloud Compute'] = df_count_compute_2019['Cloud Compute'].replace({
                                                'AWS Elastic Compute Cloud (EC2)': 'Amazon Elastic Compute Cloud (EC2)',
                                               'Google Compute Engine (GCE)': 'Google Cloud Compute Engine',
                                                'Azure Virtual Machines': 'Microsoft Azure Virtual Machines',
                                                'None': 'No / None'
                                               })

# --- 2020 Q27 ---
cloud_computing_2020 = ['Q27_A_Part_1','Q27_A_Part_2','Q27_A_Part_3','Q27_A_Part_4','Q27_A_Part_5','Q27_A_Part_6','Q27_A_Part_7','Q27_A_Part_8','Q27_A_Part_9','Q27_A_Part_10','Q27_A_Part_11','Q27_A_OTHER']
df_computing_2020 = pros_2020[cloud_computing_2020]
count_compute_2020 = pd.Series(df_computing_2020[cloud_computing_2020].squeeze().values.ravel()).value_counts()
# ---
df_count_compute_2020 = pd.DataFrame(count_compute_2020)
df_count_compute_2020 = df_count_compute_2020.reset_index()
df_count_compute_2020.columns = ['Cloud Compute', 'Counts']
# ---
df_count_compute_2020['Cloud Compute'] = df_count_compute_2020['Cloud Compute'].str.strip()
df_count_compute_2020 = df_count_compute_2020[(df_count_compute_2020['Cloud Compute'] == 'Amazon EC2') |
                      (df_count_compute_2020['Cloud Compute'] == 'Google Cloud Compute Engine') |
                     (df_count_compute_2020['Cloud Compute'] == 'Azure Cloud Services') |
                     (df_count_compute_2020['Cloud Compute'] == 'No / None')
                     ]
# ---
df_count_compute_2020['Cloud Compute'] = df_count_compute_2020['Cloud Compute'].replace({
                                                'Amazon EC2': 'Amazon Elastic Compute Cloud (EC2)',
                                               'Google Cloud Compute Engine': 'Google Cloud Compute Engine',
                                                'Azure Cloud Services': 'Microsoft Azure Virtual Machines'
                                               })



# --- 2021 Q29 ---
cloud_computing_2021 = ['Q29_A_Part_1','Q29_A_Part_2','Q29_A_Part_3','Q29_A_Part_4','Q29_A_OTHER']
df_computing_2021 = pros_2021[cloud_computing_2021]
count_compute_2021 = pd.Series(df_computing_2021[cloud_computing_2021].squeeze().values.ravel()).value_counts()
# ---
df_count_compute_2021 = pd.DataFrame(count_compute_2021)
df_count_compute_2021 = df_count_compute_2021.reset_index()
df_count_compute_2021.columns = ['Cloud Compute', 'Counts']
# ---
df_count_compute_2021['Cloud Compute'] = df_count_compute_2021['Cloud Compute'].str.strip()
df_count_compute_2021 = df_count_compute_2021[(df_count_compute_2021['Cloud Compute'] != 'Other')]

cloud_compute_df = df_count_2021.merge(df_count_2020, on = 'Cloud').merge(df_count_2019, how = 'left')
cloud_compute_df = df_count_compute_2021.merge(df_count_compute_2020, on = 'Cloud Compute').merge(df_count_compute_2019, how = 'left')
cloud_compute_df = cloud_compute_df.rename(columns = {'Cloud': 'Cloud', 'Counts_x':'2021', 'Counts_y': '2020', 'Counts': '2019'})

cloud_compute_df = cloud_compute_df.T
cloud_compute_df.columns = cloud_compute_df.iloc[0]
cloud_compute_df = cloud_compute_df.drop(cloud_compute_df.index[0])
#cloud_df.index.name = 'Cloud'
cloud_compute_df = cloud_compute_df.iloc[::-1] # reverse ro

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))
                      
cloud_compute_df.plot(kind = 'bar', 
                      ax= ax,
                      color = ('#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'))


ax.set_title("Cloud platform product usage in regular basis in kaggle survey", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.9, 0.5), fontsize=20)

plt.savefig('1.6 Cloud platform product usage in regular basis in kaggle survey.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Cloud computing product Market share ---

for_perc = cloud_compute_df
for_perc = for_perc.divide(for_perc.sum(axis=1), axis = 0)

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))
                      
for_perc.plot(kind='area', 
              stacked=True,
              ax = ax,
             color = ('#1f77b4', '#ff7f0e', '#2ca02c', '#d62728')
             )

ax.set_title("Cloud platform product usage by share", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Percentage', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.9, 0.5), fontsize=20)

plt.savefig('1.7 Cloud platform product usage by share.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- 2021 Q29_B Future preference ---

cloud_computing_in_2 = ['Q29_B_Part_1','Q29_B_Part_2','Q29_B_Part_3','Q29_B_Part_4','Q29_B_OTHER']

count_cloud_computing_in_2 = pd.Series(pros_2021[cloud_computing_in_2].squeeze().values.ravel()).value_counts()

df_count_cloud_computing_in_2 = pd.DataFrame(count_cloud_computing_in_2)
df_count_cloud_computing_in_2 = df_count_cloud_computing_in_2.reset_index()
df_count_cloud_computing_in_2.columns = ['Cloud Compute', '2021']
df_count_cloud_computing_in_2 = df_count_cloud_computing_in_2.set_index('Cloud Compute').T

# -- Plot ---

fig, ax = plt.subplots(figsize = (10,10))
df_count_cloud_computing_in_2.plot(kind = 'bar', 
                      ax= ax,
                      color = ('#ff7f0e', '#d62728', '#1f77b4', '#2ca02c','#7f7f7f'))
                    # Red = Google / Blue = Amazon / None = Green / Azure = Red

ax.set_title("Cloud computing platform that willing to become more familiar in 2 year", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.9, 0.5), fontsize=20)
ax.text(0.01, 2500, 'AWS is pushed back to the third place',
        verticalalignment='bottom',
        fontsize=12)

plt.savefig('1.8 Cloud computing platform that willing to become more familiar in 2 year.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- 2021 Q32 Big data ---

big_data_2021 = ['Q32_A_Part_1','Q32_A_Part_2',
                 'Q32_A_Part_3','Q32_A_Part_4',
                 'Q32_A_Part_5','Q32_A_Part_6',
                 'Q32_A_Part_7','Q32_A_Part_8',
                 'Q32_A_Part_9','Q32_A_Part_10',
                 'Q32_A_Part_11','Q32_A_Part_12',
                 'Q32_A_Part_13','Q32_A_Part_14',
                 'Q32_A_Part_15','Q32_A_Part_16',
                 'Q32_A_Part_17','Q32_A_Part_18',
                 'Q32_A_Part_19','Q32_A_Part_20','Q32_A_OTHER']

df_bigdata_2021 = pros_2021[big_data_2021]
count_bigdata_2021 = pd.Series(df_bigdata_2021[big_data_2021].squeeze().values.ravel()).value_counts()

df_count_bigdata_2021 = pd.DataFrame(count_bigdata_2021)
df_count_bigdata_2021 = df_count_bigdata_2021.reset_index()
df_count_bigdata_2021.columns = ['big data', 'Counts']

# --- 2020 Q29 Big data ---

big_data_2020 = ['Q29_A_Part_1','Q29_A_Part_2',
                 'Q29_A_Part_3','Q29_A_Part_4',
                 'Q29_A_Part_5','Q29_A_Part_6',
                 'Q29_A_Part_7','Q29_A_Part_8',
                 'Q29_A_Part_9','Q29_A_Part_10',
                 'Q29_A_Part_11','Q29_A_Part_12',
                 'Q29_A_Part_13','Q29_A_Part_14',
                 'Q29_A_Part_15','Q29_A_Part_16',
                 'Q29_A_Part_17','Q29_A_OTHER']

df_bigdata_2020 = pros_2020[big_data_2020]
count_bigdata_2020 = pd.Series(df_bigdata_2020[big_data_2020].squeeze().values.ravel()).value_counts()

df_count_bigdata_2020 = pd.DataFrame(count_bigdata_2020)
df_count_bigdata_2020 = df_count_bigdata_2020.reset_index()
df_count_bigdata_2020.columns = ['big data', 'Counts']

# --- 2019 Q34 Big data ---

big_data_2019 = ['Q34_Part_1','Q34_Part_2',
                 'Q34_Part_3','Q34_Part_4',
                 'Q34_Part_5','Q34_Part_6',
                 'Q34_Part_7','Q34_Part_8',
                 'Q34_Part_9','Q34_Part_10',
                 'Q34_Part_11','Q34_Part_12',
                 'Q34_OTHER_TEXT']

df_bigdata_2019 = pros_2019[big_data_2019]
count_bigdata_2019 = pd.Series(df_bigdata_2019[big_data_2019].squeeze().values.ravel()).value_counts()

df_count_bigdata_2019 = pd.DataFrame(count_bigdata_2019)
df_count_bigdata_2019 = df_count_bigdata_2019.reset_index()
df_count_bigdata_2019.columns = ['big data', 'Counts']
df_count_bigdata_2019 = df_count_bigdata_2019[(df_count_bigdata_2019['big data'] == 'MySQL') |
                                              (df_count_bigdata_2019['big data'] == 'PostgresSQL') |
                                              (df_count_bigdata_2019['big data'] == 'Microsoft SQL Server') |
                                              (df_count_bigdata_2019['big data'] == 'SQLite') |
                                              (df_count_bigdata_2019['big data'] == 'Oracle Database') |
                                              (df_count_bigdata_2019['big data'] == 'AWS Relational Database Service') |
                                              (df_count_bigdata_2019['big data'] == 'Microsoft Access') |
                                              (df_count_bigdata_2019['big data'] == 'Google Cloud SQL') |
                                              (df_count_bigdata_2019['big data'] == 'Azure SQL Database') |
                                              (df_count_bigdata_2019['big data'] == 'AWS DynamoDB')
                                             ]

# --- 2021 Clean up. Merge into AWS, GCP, and AZURE // Open source and Commercial ---

df_count_bigdata_2021['big data'] = df_count_bigdata_2021['big data'].str.strip()
df_count_bigdata_2021['open source'] = df_count_bigdata_2021['big data'].apply(open_source)
df_count_bigdata_2021['aws'] = df_count_bigdata_2021['big data'].apply(aws)
df_count_bigdata_2021['gcp'] = df_count_bigdata_2021['big data'].apply(gcp)
df_count_bigdata_2021['azure'] = df_count_bigdata_2021['big data'].apply(azure)

open_source_2021 = df_count_bigdata_2021[df_count_bigdata_2021['open source'] == 1]
open_source_2021 = open_source_2021[['big data', 'Counts']]

aws_2021 = df_count_bigdata_2021[df_count_bigdata_2021['aws'] == 1]
gcp_2021 = df_count_bigdata_2021[df_count_bigdata_2021['gcp'] == 1]
azure_2021 = df_count_bigdata_2021[df_count_bigdata_2021['azure'] == 1]
others_2021 = df_count_bigdata_2021[(df_count_bigdata_2021['aws'] != 1) & 
                                    (df_count_bigdata_2021['gcp'] != 1) & 
                                    (df_count_bigdata_2021['azure'] != 1) &
                                    (df_count_bigdata_2021['open source'] != 1)]

aws_2021['big data'].iloc[0] = 'aws'
gcp_2021['big data'].iloc[0] = 'gcp'
azure_2021['big data'].iloc[0] = 'azure'

commercial_2021 = aws_2021.iloc[:1].append([gcp_2021.iloc[:1], azure_2021.iloc[:1], others_2021])
to_drop = ['None', 'Other']
commercial_2021 = commercial_2021[~commercial_2021['big data'].isin(to_drop)]
commercial_2021 = commercial_2021[['big data', 'Counts']]

open_source_2021 = open_source_2021.merge(commercial_2021, on = 'big data', how = 'outer')
open_source_2021 = open_source_2021.rename(columns = {'Counts_x' : 'open source_2021', 'Counts_y' : 'commercial_2021'})

open_source_2021 = open_source_2021.set_index('big data').T

# --- 2020 Clean up. Merge into AWS, GCP, and AZURE // Open source and Commercial ---

df_count_bigdata_2020['big data'] = df_count_bigdata_2020['big data'].str.strip()
df_count_bigdata_2020['open source'] = df_count_bigdata_2020['big data'].apply(open_source)
df_count_bigdata_2020['aws'] = df_count_bigdata_2020['big data'].apply(aws)
df_count_bigdata_2020['gcp'] = df_count_bigdata_2020['big data'].apply(gcp)
df_count_bigdata_2020['azure'] = df_count_bigdata_2020['big data'].apply(azure)

open_source_2020 = df_count_bigdata_2020[df_count_bigdata_2020['open source'] == 1]
open_source_2020 = open_source_2020[['big data', 'Counts']]

aws_2020 = df_count_bigdata_2020[df_count_bigdata_2020['aws'] == 1]
gcp_2020 = df_count_bigdata_2020[df_count_bigdata_2020['gcp'] == 1]
azure_2020 = df_count_bigdata_2020[df_count_bigdata_2020['azure'] == 1]
others_2020 = df_count_bigdata_2020[(df_count_bigdata_2020['aws'] != 1) & 
                                    (df_count_bigdata_2020['gcp'] != 1) & 
                                    (df_count_bigdata_2020['azure'] != 1) &
                                    (df_count_bigdata_2020['open source'] != 1)]

aws_2020['big data'].iloc[0] = 'aws'
gcp_2020['big data'].iloc[0] = 'gcp'
azure_2020['big data'].iloc[0] = 'azure'

commercial_2020 = aws_2020.iloc[:1].append([gcp_2020.iloc[:1], azure_2020.iloc[:1], others_2020])
to_drop = ['None', 'Other']
commercial_2020 = commercial_2020[~commercial_2020['big data'].isin(to_drop)]
commercial_2020 = commercial_2020[['big data', 'Counts']]

open_source_2020 = open_source_2020.merge(commercial_2020, on = 'big data', how = 'outer')
open_source_2020 = open_source_2020.rename(columns = {'Counts_x' : 'open source_2020', 'Counts_y' : 'commercial_2020'})

open_source_2020 = open_source_2020.set_index('big data').T

# --- 2019 Clean up. Merge into AWS, GCP, and AZURE // Open source and Commercial ---

df_count_bigdata_2019['big data'] = df_count_bigdata_2019['big data'].str.strip()
df_count_bigdata_2019['open source'] = df_count_bigdata_2019['big data'].apply(open_source)
df_count_bigdata_2019['aws'] = df_count_bigdata_2019['big data'].apply(aws)
df_count_bigdata_2019['gcp'] = df_count_bigdata_2019['big data'].apply(gcp)
df_count_bigdata_2019['azure'] = df_count_bigdata_2019['big data'].apply(azure)

open_source_2019 = df_count_bigdata_2019[df_count_bigdata_2019['open source'] == 1]
open_source_2019 = open_source_2019[['big data', 'Counts']]

aws_2019 = df_count_bigdata_2019[df_count_bigdata_2019['aws'] == 1]
gcp_2019 = df_count_bigdata_2019[df_count_bigdata_2019['gcp'] == 1]
azure_2019 = df_count_bigdata_2019[df_count_bigdata_2019['azure'] == 1]
others_2019 = df_count_bigdata_2019[(df_count_bigdata_2019['aws'] != 1) & 
                                    (df_count_bigdata_2019['gcp'] != 1) & 
                                    (df_count_bigdata_2019['azure'] != 1) &
                                    (df_count_bigdata_2019['open source'] != 1)]

aws_2019['big data'].iloc[0] = 'aws'
gcp_2019['big data'].iloc[0] = 'gcp'
azure_2019['big data'].iloc[0] = 'azure'

commercial_2019 = aws_2019.iloc[:1].append([gcp_2019.iloc[:1], azure_2019.iloc[:1], others_2019])
to_drop = ['None', 'Other']
commercial_2019 = commercial_2019[~commercial_2019['big data'].isin(to_drop)]
commercial_2019 = commercial_2019[['big data', 'Counts']]

open_source_2019 = open_source_2019.merge(commercial_2019, on = 'big data', how = 'outer')
open_source_2019 = open_source_2019.rename(columns = {'Counts_x' : 'open source_2019', 'Counts_y' : 'commercial_2019'})

open_source_2019 = open_source_2019.set_index('big data').T

big_data_usage = pd.concat([open_source_2021, open_source_2020, open_source_2019])
big_data_usage['PostgresSQL'] = big_data_usage['PostgresSQL'].fillna(big_data_usage['PostgreSQL'])
big_data_usage = big_data_usage.drop(['PostgreSQL'], axis = 1)

# --- Clean && Combine into Open / Commercial yearly ---

big_data_usage["total"] = big_data_usage.sum(axis=1)
dummy_big_data = big_data_usage['total']
dummy_big_data_2021 = dummy_big_data[:2]
dummy_big_data_2020 = dummy_big_data[2:4]
dummy_big_data_2019 = dummy_big_data[4:6]

open_commercial_2021 = pd.DataFrame(dummy_big_data_2021)
open_commercial_2021 = open_commercial_2021.rename(index = ({'open source_2021': 'open source', 'commercial_2021': 'commercial'}), columns=({'total': '2021'}))

open_commercial_2020 = pd.DataFrame(dummy_big_data_2020)
open_commercial_2020 = open_commercial_2020.rename(index = ({'open source_2020': 'open source', 'commercial_2020': 'commercial'}), columns=({'total': '2020'}))

open_commercial_2019 = pd.DataFrame(dummy_big_data_2019)
open_commercial_2019 = open_commercial_2019.rename(index = ({'open source_2019': 'open source', 'commercial_2019': 'commercial'}), columns=({'total': '2019'}))

open_commercial_2021['2020'] = open_commercial_2020
open_commercial_2021['2019'] = open_commercial_2019
open_commercial_2021 = open_commercial_2021.iloc[:, ::-1]
open_commercial_2021 = open_commercial_2021.T

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))
open_commercial_2021.plot(kind = 'bar',
                          stacked = True,
                          ax=ax,
                         color = ['#1f77b4', '#ff7f0e'])

ax.set_title("Big data product usage", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.5), fontsize=20)

plt.savefig('1.9 Big data product usage.png', bbox_inches='tight')
plt.show()

In [ ]:
# Big data Product market shares. Open vs. Commercial

big_data_usage.drop(['total'], axis = 1, inplace = True)
big_data_usage = big_data_usage.T

big_data_usage_open_source = big_data_usage[['open source_2019', 'open source_2020', 'open source_2021']]
big_data_usage_open_source.dropna(axis = 0, how = 'all', inplace = True)

big_data_usage_commercial = big_data_usage[['commercial_2019', 'commercial_2020', 'commercial_2021']]
big_data_usage_commercial = big_data_usage_commercial.rename({
    'aws': 'AWS',
    'gcp': 'GCP',
    'azure': 'AZURE',
    'Oracle Database': 'Oracle',
    'Snowflake': 'Snowflake',
    'IBM Db2': 'IBM'
})
big_data_usage_commercial.dropna(axis = 0, how = 'all', inplace = True)

# --- Plot 3 pie charts twice for Open and Commerical ---

fig, ax = plt.subplots(figsize = (17,10))

big_data_usage_open_source.plot.pie(subplots = True, 
                                    ax=ax, 
                                    legend = False,
                                    xlabel = "", 
                                    ylabel ="", 
                                    autopct='%0.2f%%',
                                   colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
plt.savefig('2.0 Open source Big data product usage share.png', bbox_inches='tight')

fig, ax = plt.subplots(figsize = (17,10))

big_data_usage_commercial.plot.pie(subplots = True, 
                                   ax=ax, legend = False, 
                                   xlabel = "", 
                                   ylabel ="", 
                                   autopct='%0.2f%%',
                                  colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
plt.savefig('2.1 Commercial Big data product usage share.png', bbox_inches='tight')

In [ ]:
# --- Business Intelligence Tools 2021 Q34 ---

viz_2021 = ['Q34_A_Part_1','Q34_A_Part_2',
                    'Q34_A_Part_3','Q34_A_Part_4',
                    'Q34_A_Part_5','Q34_A_Part_6',
                    'Q34_A_Part_7','Q34_A_Part_8',
                    'Q34_A_Part_9','Q34_A_Part_10',
                    'Q34_A_Part_11','Q34_A_Part_12',
                    'Q34_A_Part_13','Q34_A_Part_14',
                    'Q34_A_Part_15','Q34_A_Part_16','Q34_A_OTHER']
df_viz_2021 = pd.Series(pros_2021[viz_2021].squeeze().values.ravel()).value_counts()

df_count_viz_2021 = pd.DataFrame(df_viz_2021)
df_count_viz_2021 = df_count_viz_2021.reset_index()
df_count_viz_2021.columns = ['BI tools', '2021']

df_count_viz_2021 = df_count_viz_2021[1:] # Drop None

# --- Business Intelligence Tools 2020 Q31 ---

viz_2020 = ['Q31_A_Part_1','Q31_A_Part_2',
              'Q31_A_Part_3','Q31_A_Part_4',
              'Q31_A_Part_5','Q31_A_Part_6',
              'Q31_A_Part_7','Q31_A_Part_8',
              'Q31_A_Part_9','Q31_A_Part_10',
              'Q31_A_Part_11','Q31_A_Part_12',
              'Q31_A_Part_13','Q31_A_Part_14','Q31_A_OTHER']
df_viz_2020 = pd.Series(pros_2020[viz_2020].squeeze().values.ravel()).value_counts()

df_count_viz_2020 = pd.DataFrame(df_viz_2020)
df_count_viz_2020 = df_count_viz_2020.reset_index()
df_count_viz_2020.columns = ['BI tools', '2020']

df_count_viz_2020 = df_count_viz_2020[1:] # Drop None

merged_viz = df_count_viz_2021.set_index('BI tools').combine_first(df_count_viz_2020.set_index('BI tools'))
merged_viz = merged_viz.sort_values(by=['2021'], ascending = False)

# --- Merged ---

merged_viz['2021'].iloc[0] = merged_viz['2021'].iloc[0] + merged_viz['2021'].iloc[6] # Tableau + Tableau CRM
merged_viz['2020'].iloc[4] = merged_viz['2020'].iloc[4] + merged_viz['2020'].iloc[16] # Salesforce + Einstein Analytics
merged_viz['2021'].iloc[1] = merged_viz['2021'].iloc[1] + merged_viz['2021'].iloc[7]

merged_viz = merged_viz.drop(merged_viz.index[[6,7,16]])[:-1]
merged_viz = merged_viz.T

# --- plot ---

fig, ax = plt.subplots(figsize = (10,10))

merged_viz.plot(kind = 'bar',
                ax=ax,
                color = ('#1f77b4', '#ff7f0e', '#d62728', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#17becf', '#7f7f7f','#7f7f7f', '#7f7f7f'))

ax.set_title("Commerical Business Intelligence tool usage by professionals in kaggle survey", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.8), fontsize=20)

plt.savefig('3.1 Commerical Business Intelligence tool usage by professionals in kaggle survey.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Pct Change ---
merged_viz_perc = merged_viz.pct_change(periods = 1)[1:]

# pct_change
data_bi_growth = pd.DataFrame()
data_bi_growth['total'] = merged_viz.sum(axis=1)
data_bi_growth = data_bi_growth.pct_change(periods = 1)

avg_growth_merged_2021 = round(data_bi_growth.iloc[1].mean(),2) # avg growth rate

fig, ax = plt.subplots(figsize = (10,10))

merged_viz_perc.plot(kind = 'bar',
                ax=ax,
                color = ('#1f77b4', '#ff7f0e', '#d62728', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#17becf', '#7f7f7f','#7f7f7f', '#7f7f7f'))

ax.set_title("Commerical Business Intelligence growth rate compare to previous year", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.8), fontsize=20)

plt.axhline(y=avg_growth_merged_2021, xmin=0.2, xmax= 0.7, color='black', linestyle='dotted', linewidth=5)

ax.text(-0.495, 0.52, 'Average growth rate ' +str(avg_growth_merged_2021*100) + '%')

plt.savefig('3.2 Commerical Business Intelligence growth rate compare to previous year.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Python vizualization library ---

# --- Q14 2020 ---

viz_bi_2020 =['Q14_Part_1','Q14_Part_2',
              'Q14_Part_3','Q14_Part_4',
              'Q14_Part_5','Q14_Part_6',
              'Q14_Part_7','Q14_Part_8',
              'Q14_Part_9','Q14_Part_10',
              'Q14_Part_11','Q14_OTHER',
              'Q31_A_Part_1','Q31_A_Part_2',
              'Q31_A_Part_3','Q31_A_Part_4',
              'Q31_A_Part_5','Q31_A_Part_6',
              'Q31_A_Part_7','Q31_A_Part_8',
              'Q31_A_Part_9','Q31_A_Part_10',
              'Q31_A_Part_11','Q31_A_Part_12',
              'Q31_A_Part_13','Q31_A_Part_14','Q31_A_OTHER']

lib_BI_2020 = pros_2020[viz_bi_2020].rename(columns ={
    'Q14_Part_1': 'Matplotlib',
    'Q14_Part_2': 'Seaborn',
    'Q14_Part_3': 'Plotly',
    'Q14_Part_4': 'Ggplot',
    'Q14_Part_5': 'Shiny',
    'Q14_Part_6': 'D3 js',
    'Q14_Part_7': 'Altair',
    'Q14_Part_8': 'Bokeh',
    'Q14_Part_9': 'Geoplotlib',
    'Q14_Part_10': 'Leaflet',
    'Q14_Part_11': 'None',
    'Q14_OTHER': 'Other',
    
    'Q31_A_Part_1': 'Amazon QuickSight',
    'Q31_A_Part_2': 'MS Power BI',
    'Q31_A_Part_3': 'Google Data Studio',
    'Q31_A_Part_4': 'Looker',
    'Q31_A_Part_5': 'Tableau',
    'Q31_A_Part_6': 'Salesforce',
    'Q31_A_Part_7': 'Einstein Analytics',
    'Q31_A_Part_8': 'Qlik',
    'Q31_A_Part_9': 'Domo',
    'Q31_A_Part_10': 'TIBCO',
    'Q31_A_Part_11': 'Alteryx',
    'Q31_A_Part_12': 'Sisense',
    'Q31_A_Part_13': 'SAP',
    'Q31_A_Part_14': 'None',
    'Q31_A_OTHER': 'Other'
})

lib_2020 = ['Matplotlib',
    'Seaborn',
    'Plotly',
    'Ggplot']

BI_2020 = ['Amazon QuickSight',
    'MS Power BI',
    'Google Data Studio',
    'Looker',
    'Tableau',
    'Salesforce',
    'Einstein Analytics',
    'Qlik',
    'Domo',
    'TIBCO',
    'Alteryx',
    'Sisense',
    'SAP'
    ]
# ----- library growth -----

data_viz_li_2019 = ['Q20_Part_1',
                    'Q20_Part_2',
                    'Q20_Part_3',
                    'Q20_Part_4',
                    'Q20_Part_5',
                    'Q20_Part_6',
                    'Q20_Part_7',
                    'Q20_Part_8',
                    'Q20_Part_9',
                    'Q20_Part_10',
                    'Q20_Part_11',
                    'Q20_Part_12']

df_data_viz_li_2019 = pros_2019[data_viz_li_2019]


count_data_viz_li_2019 = pd.Series(df_data_viz_li_2019[data_viz_li_2019].squeeze().values.ravel()).value_counts()
#count_bigdata_2020

df_count_data_viz_li_2019 = pd.DataFrame(count_data_viz_li_2019)
df_count_data_viz_li_2019 = df_count_data_viz_li_2019.reset_index()
df_count_data_viz_li_2019.columns = ['lib', 'Counts']
df_count_data_viz_li_2019


data_viz_li_2020 =['Q14_Part_1','Q14_Part_2',
              'Q14_Part_3','Q14_Part_4',
              'Q14_Part_5','Q14_Part_6',
              'Q14_Part_7','Q14_Part_8',
              'Q14_Part_9','Q14_Part_10',
              'Q14_Part_11','Q14_OTHER']

df_data_viz_li_2020 = pros_2020[data_viz_li_2020]


count_data_viz_li_2020 = pd.Series(df_data_viz_li_2020[data_viz_li_2020].squeeze().values.ravel()).value_counts()
#count_bigdata_2020

df_count_data_viz_li_2020 = pd.DataFrame(count_data_viz_li_2020)
df_count_data_viz_li_2020 = df_count_data_viz_li_2020.reset_index()
df_count_data_viz_li_2020.columns = ['lib', 'Counts']
df_count_data_viz_li_2020

viz_bi_2021 = ['Q14_Part_1','Q14_Part_2',
                         'Q14_Part_3','Q14_Part_4',
                         'Q14_Part_5','Q14_Part_6',
                         'Q14_Part_7','Q14_Part_8',
                         'Q14_Part_9','Q14_Part_10',
                         'Q14_Part_11','Q14_OTHER']

df_data_viz_li_2021 = pros_2021[viz_bi_2021]


count_data_viz_li_2021 = pd.Series(df_data_viz_li_2021[viz_bi_2021].squeeze().values.ravel()).value_counts()
#count_bigdata_2020

df_count_data_viz_li_2021 = pd.DataFrame(count_data_viz_li_2021)
df_count_data_viz_li_2021 = df_count_data_viz_li_2021.reset_index()
df_count_data_viz_li_2021.columns = ['lib', 'Counts']

# ---------------

df_count_data_viz_li_2021 = df_count_data_viz_li_2021.merge(df_count_data_viz_li_2020, on = 'lib', how = 'outer')
df_count_data_viz_li_2021 = df_count_data_viz_li_2021.merge(df_count_data_viz_li_2019, on = 'lib', how = 'outer')
df_count_data_viz_li_2021 = df_count_data_viz_li_2021.rename(columns = {'Counts_x' : '2021 ', 'Counts_y' : '2020', 'Counts': '2019'})
df_count_data_viz_li_2021.at[8,'lib'] = 'D3.js'
df_count_data_viz_li_2021.at[8,'2019'] = df_count_data_viz_li_2021.at[12,'2019']
df_count_data_viz_li_2021 = df_count_data_viz_li_2021[:-1]
df_count_data_viz_li_2021 = df_count_data_viz_li_2021.set_index('lib').T
df_count_data_viz_li_2021 = df_count_data_viz_li_2021[::-1]
df_count_data_viz_li_2021 = df_count_data_viz_li_2021.drop(columns = ['None'])

# --- Plot ---

fig, ax = plt.subplots(figsize=(10,10))

df_count_data_viz_li_2021.plot(kind='bar', 
                               ax = ax,
                              color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
                              )

ax.set_title("Visualization Library usage by professionals in kaggle survey", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.8), fontsize=20)

plt.savefig('3.3 Visualization Library usage by professionals in kaggle survey.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Library growth ---

df_count_data_viz_li_2021_perc = df_count_data_viz_li_2021.pct_change(periods = 1)[1:]

# pct_change
data_viz_growth = pd.DataFrame()
data_viz_growth['total'] = df_count_data_viz_li_2021.sum(axis=1)
data_viz_growth_perc = data_viz_growth.pct_change(periods = 1)

avg_growth_2020 = round(data_viz_growth_perc.iloc[1].mean(),2)
avg_growth_2021 = round(data_viz_growth_perc.iloc[2].mean(),2)

# --- Plot ---

fig, ax = plt.subplots(figsize=(10,10))

df_count_data_viz_li_2021_perc.plot(kind = 'bar', 
                                    ax=ax,
                                   color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("Visualization library usage growth rate compare to previous year", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.6, 0.8), fontsize=20)

plt.axhline(y=0, xmin=-1, xmax= 2, color='red', linestyle='dotted', linewidth=5)
plt.axhline(y=avg_growth_2020, xmin=0.05, xmax= 0.45, color='black', linestyle='dotted', linewidth=5)
plt.axhline(y=avg_growth_2021, xmin=0.55, xmax= 0.95, color='black', linestyle='dotted', linewidth=5)

plt.rc('font', size=16)

ax.text(0.26, 0.08, 'Avg growth rate ' +str(avg_growth_2020*100) + '%')
ax.text(0.25, 0.35, 'Avg growth rate '+ str(avg_growth_2021*100) + '%')

plt.savefig('3.4 Visualization library usage growth rate compare to previous year.png', bbox_inches='tight')
plt.show()
plt.rcParams['font.size'] = 20

In [ ]:
# --- Machine learning tools ---
# --- Google cloud Speect-to-Text, Google Cloud Natural Language, Google Cloud Vision, Google Cloud Translation are assumed under Google Vertex AI ---

# --- 2019 Q32 ML tools ---

ml_product_2019 = ['Q32_Part_1','Q32_Part_2','Q32_Part_3','Q32_Part_4','Q32_Part_5','Q32_Part_6','Q32_Part_7','Q32_Part_8','Q32_Part_9','Q32_Part_10','Q32_Part_11','Q32_Part_12']
df_ml_product_2019 = pros_2019[ml_product_2019]
count_ml_product_2019 = pd.Series(df_ml_product_2019[ml_product_2019].squeeze().values.ravel()).value_counts()

df_count_ml_2019 = pd.DataFrame(count_ml_product_2019)
df_count_ml_2019 = df_count_ml_2019.reset_index()
df_count_ml_2019.columns = ['ML engine', 'Counts']

# --- 2020 Q28 ML tools ---

ml_product_2020 = ['Q28_A_Part_1','Q28_A_Part_2','Q28_A_Part_3','Q28_A_Part_4','Q28_A_Part_5','Q28_A_Part_6','Q28_A_Part_7','Q28_A_Part_8','Q28_A_Part_9','Q28_A_Part_10']
df_ml_product_2020 = pros_2020[ml_product_2020]
count_ml_product_2020 = pd.Series(df_ml_product_2020[ml_product_2020].squeeze().values.ravel()).value_counts()

df_count_ml_2020 = pd.DataFrame(count_ml_product_2020)
df_count_ml_2020 = df_count_ml_2020.reset_index()
df_count_ml_2020.columns = ['ML engine', 'Counts']

# --- 2021 Q31 ML tools ---

ml_product_2021 = ['Q31_A_Part_1','Q31_A_Part_2','Q31_A_Part_3','Q31_A_Part_4','Q31_A_Part_5','Q31_A_Part_6','Q31_A_Part_7','Q31_A_Part_8','Q31_A_Part_9','Q31_A_OTHER']
df_ml_product_2021 = pros_2021[ml_product_2021]
count_ml_product_2021 = pd.Series(df_ml_product_2021[ml_product_2021].squeeze().values.ravel()).value_counts()

df_count_ml_2021 = pd.DataFrame(count_ml_product_2021)
df_count_ml_2021 = df_count_ml_2021.reset_index()
df_count_ml_2021.columns = ['ML engine', 'Counts']

# --- Products are merged into one big category ---

# ex1) google vision, google NLP - > Google Cloud Vertex AI
# ex2) Amazon forecast, recognition - > Amazon SageMaker

df_count_ml_2019 = df_count_ml_2019.drop(df_count_ml_2019.index[[0,5,7,8,9,11]])

df_count_ml_2020 = df_count_ml_2020.drop(df_count_ml_2020.index[[0,4,5,6,7,8,9]])

df_count_ml_2021 = df_count_ml_2021.drop(df_count_ml_2021.index[[0, 7]])

engine_df = df_count_ml_2021.merge(df_count_ml_2020, on = 'ML engine', how = 'outer').merge(df_count_ml_2019, how = 'outer')
engine_df = engine_df.rename(columns = {'Counts_x' : '2021', 'Counts_y' : '2020', 'Counts' : '2019'})
engine_df['ML engine'] = engine_df['ML engine'].str.strip()
engine_df.at[8, 'ML engine'] = 'Google Cloud Vertex AI'
engine_df.at[9, 'ML engine'] = 'Google Cloud Vertex AI'
engine_df = engine_df.groupby(['ML engine']).sum()
engine_df = engine_df.sort_values(by=['2021'], ascending = False).reset_index()

engine_df = engine_df.set_index('ML engine').T[::-1]
columns_clean = ['Amazon SageMaker','Azure Machine Learning Studio','Databricks','Google Cloud Vertex AI','DataRobot','Rapidminer','Alteryx','Dataiku']
engine_df = engine_df[columns_clean]

dummy_df = engine_df[['Amazon SageMaker','Azure Machine Learning Studio','Google Cloud Vertex AI']]
dummy_df['multiplier'] = None
dummy_df['multiplier'] = dummy_df.sum(axis=1).pct_change(periods = 1)
growth_multiplier_2020 = dummy_df['multiplier'].iloc[1]
growth_multiplier_2021 = dummy_df['multiplier'].iloc[2]

for col in ['Databricks', 'DataRobot', 'Rapidminer', 'Alteryx', 'Dataiku']:
    engine_df[col].iloc[1] = engine_df[col].iloc[2]/(1+growth_multiplier_2021)
    engine_df[col].iloc[0] = engine_df[col].iloc[1]/(1+growth_multiplier_2020)

engine_df = engine_df.round(0)

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))

engine_df.plot(kind = 'bar',
                ax=ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f'])

ax.set_title("Managed ML product usage on regular basis", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.8, 0.6), fontsize=20)
ax.text(2, 740, 'Google pushed back')
ax.text(1, 740, 'Google led')

plt.annotate('* The number of Databricks, Datarobot, RapidMiner, Alteryx and Dataiku in 2019 and 2020 were derived from the yearly avg industry growth', 
             (0,0), 
             (-50,-95), 
             fontsize=8, 
             xycoords='axes fraction', 
             textcoords='offset points', va='top')


plt.annotate("* Google vertex AI is launched in 2021. Google vertex AI in 2019 and 2020 is the sum of google's listed products. This applies to AWS and Azure too", 
             (0,0), 
             (-50,-105), 
             fontsize=8, 
             xycoords='axes fraction', 
             textcoords='offset points', va='top')

plt.savefig('4.1 Managed ML product usage on regular basis.png', bbox_inches='tight')
plt.show()

In [ ]:
managed_ml_2021_2_year = ['Q31_B_Part_1','Q31_B_Part_2',
                          'Q31_B_Part_3','Q31_B_Part_4',
                          'Q31_B_Part_5','Q31_B_Part_6',
                          'Q31_B_Part_7','Q31_B_Part_8',
                          'Q31_B_Part_9','Q31_B_OTHER']

df_managed_ml_2021_2_year = pros_2021[managed_ml_2021_2_year]

count_managed_ml_2021_2_year = pd.Series(df_managed_ml_2021_2_year[managed_ml_2021_2_year].squeeze().values.ravel()).value_counts()

df_count_managed_ml_2021_2_year = pd.DataFrame(count_managed_ml_2021_2_year)
df_count_managed_ml_2021_2_year = df_count_managed_ml_2021_2_year.reset_index()
df_count_managed_ml_2021_2_year.columns = ['Managed ML', '2021']
df_count_managed_ml_2021_2_year = df_count_managed_ml_2021_2_year.set_index('Managed ML').T
df_count_managed_ml_2021_2_year.columns = df_count_managed_ml_2021_2_year.columns.str.strip()
df_count_managed_ml_2021_2_year = df_count_managed_ml_2021_2_year[['Amazon SageMaker','Azure Machine Learning Studio',
                                 'Google Cloud Vertex AI','Databricks', 'DataRobot', 'Rapidminer', 'Alteryx', 'Dataiku']]
df_count_managed_ml_2021_2_year = df_count_managed_ml_2021_2_year.T.sort_values(by= '2021', ascending = False).T

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))

df_count_managed_ml_2021_2_year.plot(kind = 'bar',
                ax=ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f', '#7f7f7f'])

ax.set_title("Willingness to become more familiar in the next 2 years", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.8, 0.6), fontsize=20)
ax.text(-0.42, 2350, 'Google is back to 1st, Azure just behind')

plt.savefig('4.2 Willingness to become more familiar in the next 2 years.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- AutoML ---

# --- 2021 Q21 AutoML---

autoML_2021 = ['Q37_A_Part_1','Q37_A_Part_2','Q37_A_Part_3','Q37_A_Part_4','Q37_A_Part_5','Q37_A_Part_6','Q37_A_Part_7','Q37_A_OTHER']
df_autoML_2021 = pros_2021[autoML_2021]
count_autoML_2021 = pd.Series(df_autoML_2021[autoML_2021].squeeze().values.ravel()).value_counts()

df_count_autoML_2021 = pd.DataFrame(count_autoML_2021)
df_count_autoML_2021 = df_count_autoML_2021.reset_index()
df_count_autoML_2021.columns = ['AutoML', 'Counts']

# --- 2020 Q34 AutoML---

autoML_2020 = ['Q34_A_Part_1','Q34_A_Part_2','Q34_A_Part_3','Q34_A_Part_4','Q34_A_Part_5','Q34_A_Part_6','Q34_A_Part_7','Q34_A_Part_8','Q34_A_Part_9','Q34_A_Part_10','Q34_A_Part_11','Q34_A_OTHER']
df_autoML_2020 = pros_2020[autoML_2020]
count_autoML_2020 = pd.Series(df_autoML_2020[autoML_2020].squeeze().values.ravel()).value_counts()

df_count_autoML_2020 = pd.DataFrame(count_autoML_2020)
df_count_autoML_2020 = df_count_autoML_2020.reset_index()
df_count_autoML_2020.columns = ['AutoML', 'Counts']

# --- 2019 Q33 AutoML---

autoML_2019 = ['Q33_Part_1','Q33_Part_2','Q33_Part_3','Q33_Part_4','Q33_Part_5','Q33_Part_6','Q33_Part_7','Q33_Part_8','Q33_Part_9','Q33_Part_10','Q33_Part_11','Q33_Part_12']
df_autoML_2019 = pros_2019[autoML_2019]
count_autoML_2019 = pd.Series(df_autoML_2019[autoML_2019].squeeze().values.ravel()).value_counts()

df_count_autoML_2019 = pd.DataFrame(count_autoML_2019)
df_count_autoML_2019 = df_count_autoML_2019.reset_index()
df_count_autoML_2019.columns = ['AutoML', 'Counts']

# --- AutoML merged ---

autoML_df = df_count_autoML_2021.merge(df_count_autoML_2020, on = 'AutoML', how = 'outer').merge(df_count_autoML_2019, how = 'outer')
autoML_df = autoML_df.rename(columns = {'Counts_x' : '2021', 'Counts_y' : '2020', 'Counts' : '2019'})
autoML_df = autoML_df.drop(autoML_df.index[[0,7,8,9,10,12,13,14,15]])
autoML_df.at[16, 'AutoML'] = 'Google Cloud AutoML'
autoML_df.at[11, 'AutoML'] = 'H2O Driverless AI'
autoML_df['AutoML'] = autoML_df['AutoML'].str.strip()
autoML_df = autoML_df.groupby(['AutoML']).sum()
autoML_df = autoML_df.sort_values(by=['2021'], ascending = False)

autoML_df = autoML_df.T
autoML_df = autoML_df[::-1]

dummy_df = autoML_df[['DataRobot AutoML','Databricks AutoML','Google Cloud AutoML','H2O Driverless AI']]
dummy_df['total'] = dummy_df.sum(axis=1)
dummy_df['total'] = dummy_df['total'].pct_change(periods=1)
autoML_growth_2020 = dummy_df['total'].iloc[1]
autoML_growth_2021 = dummy_df['total'].iloc[2]

sage_azure = ['Amazon Sagemaker Autopilot', 'Azure Automated Machine Learning']

for col in sage_azure:
    autoML_df[col].iloc[1] = autoML_df[col].iloc[2]/(1+autoML_growth_2021)
    autoML_df[col].iloc[0] = autoML_df[col].iloc[1]/(1+autoML_growth_2020)
    
autoML_df = autoML_df.round(0)

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))

autoML_df.plot(kind = 'bar',
                ax=ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("AutoML usage from 2019 to 2021", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.8, 0.6), fontsize=20)

plt.annotate('* Numbers in Amazon Sagemaker Autopilot and Azure Automated Machine Learning in 2019 and 2020 were derived from the yearly avg industry growth', 
             (0,0), 
             (-50,-95), 
             fontsize=8, 
             xycoords='axes fraction', 
             textcoords='offset points', va='top')

plt.savefig('5.1 AutoML usage from 2019 to 2021.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- AutoML future Preference ---

# --- 2021 Q31 AutoML ---

managed_ml_2021_2_year = ['Q31_B_Part_1','Q31_B_Part_2',
                          'Q31_B_Part_3','Q31_B_Part_4',
                          'Q31_B_Part_5','Q31_B_Part_6',
                          'Q31_B_Part_7','Q31_B_Part_8',
                          'Q31_B_Part_9','Q31_B_OTHER']

automl_in_2 = ['Q37_B_Part_1',
               'Q37_B_Part_2',
               'Q37_B_Part_3',
               'Q37_B_Part_4',
               'Q37_B_Part_5',
               'Q37_B_Part_6',
               'Q37_B_Part_7',
               'Q37_B_OTHER']

df_automl_in_2 = pros_2021[automl_in_2]

count_df_automl_in_2 = pd.Series(df_automl_in_2[automl_in_2].squeeze().values.ravel()).value_counts()

df_count_df_automl_in_2 = pd.DataFrame(count_df_automl_in_2)
df_count_df_automl_in_2 = df_count_df_automl_in_2.reset_index()
df_count_df_automl_in_2.columns = ['Auto ML', '2021']
df_count_df_automl_in_2 = df_count_df_automl_in_2.set_index('Auto ML').T
df_count_df_automl_in_2.columns = df_count_df_automl_in_2.columns.str.strip()
df_count_df_automl_in_2 = df_count_df_automl_in_2[['Google Cloud AutoML','Azure Automated Machine Learning', 'Amazon Sagemaker Autopilot', 'Databricks AutoML', 'DataRobot AutoML', 'H2O Driverless AI']]

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))

df_count_df_automl_in_2.plot(kind = 'bar',
                ax=ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("AutoML: Willingness to become more familiar in the next 2 years", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.8, 0.6), fontsize=20)

plt.savefig('5.2 AutoML: Willingness to become more familiar in the next 2 years.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Special hardware usage ---

# 2021 Q12 Special hardware usage 

hardware_2021 = ['Q12_Part_1','Q12_Part_2','Q12_Part_3','Q12_Part_4','Q12_Part_5','Q12_OTHER']
hardware_2021_df = pros_2021[hardware_2021]
count_hardware_2021 = pd.Series(hardware_2021_df[hardware_2021].squeeze().values.ravel()).value_counts()

# 2020 Q12 Special hardware usage 

hardware_2020 = ['Q12_Part_1','Q12_Part_2','Q12_Part_3','Q12_OTHER']
hardware_2020_df = pros_2020[hardware_2020]
count_hardware_2020 = pd.Series(hardware_2020_df[hardware_2020].squeeze().values.ravel()).value_counts()

# 2021 Q19 Special hardware usage 

hardware_2019 = ['Q21_Part_1','Q21_Part_2','Q21_Part_3','Q21_Part_4','Q21_Part_5','Q21_OTHER_TEXT']
hardware_2019_df = pros_2019[hardware_2019]
count_hardware_2019 = pd.Series(hardware_2019_df[hardware_2019].squeeze().values.ravel()).value_counts()

df_count_hardware_2021 = pd.DataFrame(count_hardware_2021)
df_count_hardware_2021 = df_count_hardware_2021.reset_index()
df_count_hardware_2021.columns = ['manage', '2021']

df_count_hardware_2020 = pd.DataFrame(count_hardware_2020)
df_count_hardware_2020 = df_count_hardware_2020.reset_index()
df_count_hardware_2020.columns = ['manage', '2020']

df_count_hardware_2019 = pd.DataFrame(count_hardware_2019)
df_count_hardware_2019 = df_count_hardware_2019.reset_index()
df_count_hardware_2019.columns = ['manage', '2019']

df_count_hardware_2019 = df_count_hardware_2019.iloc[1:6]
df_count_hardware_2019 = df_count_hardware_2019.set_index('manage').T[['GPUs', 'TPUs','Other']]
df_count_hardware_2020 = df_count_hardware_2020.set_index('manage').T[['GPUs', 'TPUs', 'Other']]

df_count_hardware_2021 = df_count_hardware_2021.set_index('manage').T
df_count_hardware_2021.columns = df_count_hardware_2021.columns.str.strip()
df_count_hardware_2021 = df_count_hardware_2021[['NVIDIA GPUs','Google Cloud TPUs','Other','AWS Inferentia Chips','AWS Trainium Chips']]
df_count_hardware_2021 = df_count_hardware_2021.rename(columns = {'NVIDIA GPUs': 'GPUs',
                                                                    'Google Cloud TPUs': 'TPUs',
                                                                    'Other': 'Other',
                                                                    'AWS Inferentia Chips': 'AWS Inferentia',
                                                                    'AWS Trainium Chips': 'AWS Trainium'})

# Merge all three

df_hardware_merged = df_count_hardware_2019.append(df_count_hardware_2020).append(df_count_hardware_2021).fillna(0)

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))

df_hardware_merged.plot(kind = 'bar',
                ax=ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("Special hardware usage trend", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.5, 0.6), fontsize=20)
ax.annotate("AWS chips added in 2021", xy=(2.1, 1000), xytext=(2.3, 1500),
            arrowprops=dict(facecolor='black', shrink=0.05),
            )

plt.savefig('6.1 Special hardware usage trend.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Special hardware // Company Spending ---

# --- Special hardware // Company Spending 2020 Q12 ---

large_spender_2020 = pros_2020.loc[pros_2020['Q25'].isin(['$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', '$100,000 or more ($USD)'])]
hardware_2020 = ['Q12_Part_1','Q12_Part_2','Q12_Part_3','Q12_OTHER']

spender_hardware_2020 = pd.DataFrame()

money_array = ['$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', '$100,000 or more ($USD)']

for idx, comp_size in enumerate(money_array):
    spender_comp_size = large_spender_2020.loc[large_spender_2020['Q25'] == comp_size]
    idx = pd.Series(spender_comp_size[hardware_2020].squeeze().values.ravel()).value_counts()
    spender_hardware_2020[comp_size] = idx
    
spender_hardware_2020 = spender_hardware_2020.rename(columns = {'$100,000 or more ($USD)': '100,000 +'})
spender_hardware_2020 = spender_hardware_2020.T[['GPUs','TPUs', 'Other']]

# --- Special hardware // Company Spending 2021 Q26 ---

large_spender_2021 = pros_2021.loc[pros_2021['Q26'].isin(['$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', '$100,000 or more ($USD)'])]

hardware_2021 = ['Q12_Part_1','Q12_Part_2','Q12_Part_3','Q12_Part_4','Q12_Part_5','Q12_OTHER']

spender_hardware_2021 = pd.DataFrame()

money_lst = ['$1-$99', '$100-$999', '$1000-$9,999', '$10,000-$99,999', '$100,000 or more ($USD)']

for idx, comp_size in enumerate(money_lst):
    spender_comp_size = large_spender_2021.loc[large_spender_2021['Q26'] == comp_size]
    idx = pd.Series(spender_comp_size[hardware_2021].squeeze().values.ravel()).value_counts()
    spender_hardware_2021[comp_size] = idx
    
spender_hardware_2021 = spender_hardware_2021.rename(columns = {'$100,000 or more ($USD)': '100,000 +'})
spender_hardware_2021 = spender_hardware_2021[1:].T

# --- Plot (1) ---

fig, ax = plt.subplots(figsize = (10,10))

spender_hardware_2020.plot(kind = 'bar',
                ax=ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("Special hardware usage by spending size in 2020", fontsize ='large', pad=20)
ax.set_xlabel('Money spent', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.3, 0.6), fontsize=20)

plt.savefig('6.2 Special hardware usage trend.png', bbox_inches='tight')
plt.show()

# --- Plot (2) ---

fig, ax = plt.subplots(figsize = (10,10))

spender_hardware_2021.plot(kind = 'bar',
                ax=ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("Special hardware usage by spending size in 2021", fontsize ='large', pad=20)
ax.set_xlabel('Money spent', fontsize='medium', labelpad=20)
ax.set_ylabel('Number of respondents', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.6, 0.6), fontsize=20)

plt.savefig('6.3 Special hardware usage trend.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- NVIDIA Earnings ---

nvda_earnings = pd.read_excel('../input/stonks-data/nvda_earnings.xlsx')
nvda_earnings = nvda_earnings.rename(columns = {'Unnamed: 0': 'Revenue'})

df = nvda_earnings.T
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.drop('Total', axis = 1, inplace = True)

# --- Plot ---

fig, ax = plt.subplots(figsize = (10, 10))

df.plot(kind = 'bar',
        stacked=True, 
        ax = ax,
       color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("NVIDIA Earnings", fontsize ='large', pad=20)
ax.set_xlabel('Adj Fiscal Year', fontsize='medium', labelpad=20)
ax.set_ylabel('in Million USD', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.65, 0.6), fontsize=20)

plt.annotate('',
ha = 'center', va = 'bottom',
xytext = (7.8, 1800),
xy = (12.3, 3800),
arrowprops = { 'facecolor' : 'red', 'shrink' : 0.05 })

plt.annotate('Data Center Revenue Explosion Begins',
        fontsize = 14,
        ha = 'center', va = 'bottom',
        xytext = (8, 5000),
        xy = (8, 3200),
        arrowprops = { 'facecolor' : 'black', 'shrink' : 0.05 })


#plt.annotate('Data Center revenue growth 100% YoY in 2020', xytext = (3, 6000), xy = (8, 6000), fontsize = 22, color = 'red', )
plt.axhline(y=3200, linestyle='dashed', linewidth=2, color = 'black') 
plt.annotate('Source: Nvidia', (0,0), (-80,-95), fontsize=8, 
             xycoords='axes fraction', textcoords='offset points', va='top')

plt.rc('font', size=11) 
for c in (ax.containers):
    ax.bar_label(c, label_type='center')
    
plt.rcParams['font.size'] = 20

plt.savefig('7.1 NVIDIA Earnings.png', bbox_inches='tight')
plt.show()

In [ ]:
# --- Estimated Public Cloud's Spending on GPU infrastructure ---

# Current market share: AWS = 31%, Azure = 22%, and GCP = 8%
# Jensen Huang said 51% of data center revenue was coming from public cloud

data_center_segment = nvda_earnings.iloc[[2]]
data_center_segment = data_center_segment.set_index(['Revenue'])
data_center_aws =  data_center_segment.copy() * 0.31
data_center_aws = data_center_aws.rename(index = {'data center': 'AWS'})

data_center_azure = data_center_segment.copy() * 0.22
data_center_azure = data_center_azure.rename(index = {'data center': 'Azure'})

data_center_gcp = data_center_segment.copy() * 0.08
data_center_gcp = data_center_gcp.rename(index = {'data center': 'GCP'})

data_center_others = data_center_segment.copy() * 0.39
data_center_others = data_center_others.rename(index = {'data center': 'Other Enterprises'})

df_data_center_others = pd.concat([data_center_aws, data_center_azure, data_center_gcp, data_center_others])

df_data_center_others.iloc[2, 0] = 0
df_data_center_others.iloc[2, 1] = 0
df_data_center_others.iloc[2, 2] = 0 
df_data_center_others = df_data_center_others.T
df_data_center_others = df_data_center_others[['AWS', 'Azure', 'GCP']]*0.51

# --- Plot ---

fig, ax = plt.subplots(figsize = (10, 10))

df_data_center_others.plot(kind = 'bar',
        ax = ax,
       color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("Estimated Nvidia's revenue from the big 3", fontsize ='large', pad=20)
ax.set_xlabel('Adj Fiscal Year', fontsize='medium', labelpad=20)
ax.set_ylabel('in Million USD', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.3, 0.6), fontsize=20)

plt.axvline(x=2.5, linestyle='dashed', linewidth=2, color = 'black') 
plt.annotate('Google disclosed GCP as of 2019', xytext = (2.6, 600), xy = (8, 600), fontsize = 14, color = 'black')

plt.annotate('Source: NVIDIA, https://www.parkmycloud.com/', (0,0), (-80,-120), fontsize=8, 
             xycoords='axes fraction', textcoords='offset points', va='top')

plt.savefig("7.2 Estimated Nvidia's revenue from the big 3.png", bbox_inches='tight')
plt.show()

In [ ]:
nvda_stocks = pd.read_excel('../input/nvda-market-cap/nvda_marketcap.xlsx')
nvda_stocks = nvda_stocks.head(1000)
nvda_stocks = nvda_stocks.rename(columns = {'NVDA.O (Fundamental)': 'Market Cap'})
nvda_stocks = nvda_stocks.iloc[1:, :]
nvda_stocks = nvda_stocks.drop('NVDA.O', axis = 1)
nvda_stocks = nvda_stocks.set_index('Date')
nvda_stocks = nvda_stocks.div(1000000000)

fig, ax = plt.subplots(figsize=(18,6))
nvda_stocks.plot(ax=ax,
                    color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("NVDA Market cap", fontsize ='large', pad=20)
ax.set_xlabel('Date', fontsize='medium', labelpad=20)
ax.set_ylabel('Market cap (Billions)', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.2, 0.6), fontsize=20)

plt.annotate('Gaming, Crypto and Cloud Beneficiary', (0,0), (500,150), fontsize=15, 
             xycoords='axes fraction', textcoords='offset points', va='top')

plt.annotate('Accelerated AI competition', (0,0), (800,220), fontsize=15, 
             xycoords='axes fraction', textcoords='offset points', va='top')

plt.annotate('META', (0,0), (950,280), fontsize=15, 
             xycoords='axes fraction', textcoords='offset points', va='top')

plt.annotate('Source: Refinitiv', (0,0), (-80,-80), fontsize=8, 
             xycoords='axes fraction', textcoords='offset points', va='top')

plt.savefig("NVDIA_market_cap.png", bbox_inches='tight')
plt.show()

In [ ]:
# --- Data Science professional distribution ---

# Get data from 2021

industry_2021 = data_2021[data_2021['Q20'].notna()]
c = industry_2021['Q20'].value_counts(normalize=True).rename_axis('industry').reset_index(name='counts')
#c =industry_2021['Q20'].value_counts().rename_axis('industry').reset_index(name='counts')

# Get data from 2018

industry_2018 = data_2018[data_2018['Q7'] != 'I am a student']
industry_2018 = industry_2018[industry_2018['Q7'].notna()]
d = industry_2018['Q7'].value_counts(normalize=True).rename_axis('industry').reset_index(name='counts')
#d = industry_2018['Q7'].value_counts().rename_axis('industry').reset_index(name='counts')

# compute the industry

k = pd.merge(left = d, right = c, on = 'industry')
k = k.rename(columns = {'counts_x': '2018', 'counts_y': '2021'})
k = k.sort_values(by=['2021'], ascending=False)

# compute the difference
diff_industry = k.copy()
diff_industry['dff'] = k['2021'] - k['2018']

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize = (20,10))

gs = gridspec.GridSpec(1, 2, wspace=0.2,hspace=0.5, width_ratios=[3,1]) 

ax1 = plt.subplot(gs[0])
k.plot.barh(x = "industry", 
            ax= ax1,
           color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
ax1.set(title = "Data Science professionals distribution by Industry. 2018 vs 2021",
      xlabel = "Percentage (Total sum = 1.0)",
      ylabel = "Industry")
ax1.invert_yaxis()

ax2 = plt.subplot(gs[1])
diff_industry['dff'].plot(kind='barh', x = 'industry', ax = ax2,
                    color=(diff_industry['dff'] > 0).map({True: 'g',
                                                    False: 'r'}))
ax2.set(title = "Change",
      xlabel = "Percentage",
      ylabel = "Industry")
plt.xticks(rotation=45)
ax2.set_yticks([])

plt.gca().invert_yaxis()

plt.savefig("8.1 Data Science professionals distribution by Industry. 2018 vs 2021.png", bbox_inches='tight')
plt.show()

In [ ]:
# Exhibit 1.1 Data Science distribution by company size 2021.

test = get_professionals(data_2021, 'Q5')
#print(len(test))
test = test[test['Q21'].notna()]
#print(len(test))

category_test = test.groupby(['Q20', 'Q21']).size()
#category_test.plot(kind='bar')
new_df = category_test.to_frame(name = 'size').reset_index()
new_df_2= pd.pivot(
    data = new_df,
    index = 'Q20',
    columns = 'Q21',
    values = 'size',
)
new_df_2.index.names = ['Industry']
new_df_2.columns.names = ['Company Size']

columns_order = ['0-49 employees', '50-249 employees', '250-999 employees', '1000-9,999 employees','10,000 or more employees']

new_df_2 = new_df_2.reindex(columns = columns_order)
new_df_2['total'] = new_df_2[columns_order].sum(axis = 1)
new_df_2 = new_df_2.sort_values(by = 'total', ascending = False)
new_df_2 = new_df_2.drop(columns='total')

# -----


new_df_2['total'] = new_df_2[columns_order].sum(axis = 1)
new_df_2 = new_df_2.sort_values(by = 'total', ascending = False)
new_df_2 = new_df_2.drop(columns = 'total')
res = new_df_2.div(new_df_2.sum(axis=1), axis = 0)

# --------------------- plot ---------------------
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize = (20,10))

gs = gridspec.GridSpec(1, 2, wspace=0.2,hspace=0.5, width_ratios=[3,1]) 

ax1 = plt.subplot(gs[0])
new_df_2.plot(use_index = True,  
              kind='barh', 
              stacked=True, 
              ax = ax1,
              color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
              )

ax1.set(title = "DS professional distibution by company size across different industry 2021",
      xlabel = "the number of respondents",
      ylabel = "Industry")
plt.gca().invert_yaxis()

ax2 = plt.subplot(gs[1])
res.plot(use_index = True,  
              kind='barh', 
              stacked=True, 
              ax = ax2,
         color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
              )
ax2.set(title='Company Size portion',
      xlabel = "Percentage",
      ylabel = " ")

#plt.legend(title = "Company Size", bbox_to_anchor=(1.04,1), loc="upper left")
ax2.get_legend().remove()
ax2.set_yticks([])

plt.gca().invert_yaxis()

plt.savefig("8.2 DS professional distibution by company size across different industry 2021.png", bbox_inches='tight')
plt.show()

In [ ]:
pd.options.mode.chained_assignment = None

# Industry and professionals heatmap

# ----------------
# Job title filter
# ----------------

job_title = {'Other':'Other',
     'Product Manager': 'Product/Project Manager',
 'Program/Project Manager':'Product/Project Manager',
 'Principal Investigator':'Product/Project Manager',
 'Chief Officer':'Product/Project Manager',
 'Manager':'Product/Project Manager',
 'Software Developer/Software Engineer': 'Software Engineer',
 'Operations Research Practitioner': 'Research Scientist',
 'Computer Scientist': 'Research Scientist',
 'Scientist/Researcher': 'Research Scientist',
 'Researcher': 'Research Scientist',
 'Data Scientist': 'Data Scientist',
     'Business Analyst': 'Business Analyst',
     'Engineer': 'Other',
     'DBA/Database Engineer': 'DBA/Database Engineer',
     'Data Analyst':'Data Analyst',
     'Machine Learning Engineer': 'Machine Learning Engineer',
     'Statistician':'Statistician',
     'Predictive Modeler':'Research Scientist',
     'Programmer': 'Software Engineer',
     'Data Miner': 'Data Engineer',
     'Consultant': 'Other',
     'Research Assistant': 'Research Scientist',
     'Chief Officer':'Product/Project Manager',
     'Data Engineer':'Data Engineer',
     'Developer Advocate': 'Developer Relations/Advocacy',
     'Marketing Analyst': 'Business Analyst',
     'Data Analyst': 'Data Analyst',
     'Software Engineer': 'Software Engineer',
     'Research Scientist': 'Research Scientist',
     'Data Journalist': 'Data Analyst',
     'Salesperson':'Developer Relations/Advocacy',
     'Product/Project Manager': 'Product/Project Manager',
     'Developer Relations/Advocacy': 'Developer Relations/Advocacy'
}

# -----------------------------------------
# Heatmap of job title within industry 2021
# -----------------------------------------

workforce_2021 = get_professionals(data_2021, 'Q5')
professional_2021 = workforce_2021[workforce_2021['Q20'].notna()]
professional_2021['Q5'] = professional_2021['Q5'].map(job_title)
industry_2021 = professional_2021['Q20'].unique()
df_2021 = professional_2021[['Q5','Q20']]

temp_d = {}

for industry in industry_2021:
    temp_df = df_2021[df_2021['Q20'] == industry]
    temp_dict = dict(temp_df['Q5'].value_counts())
    temp_d[industry] = temp_dict

def sorted_simple_dict(d):
    return {k: v for k, v in sorted(d.items())}

def sorted_once_nested_dict(d):
    return {k: sorted_simple_dict(v) for k, v in sorted(d.items())}

temp_d = sorted_once_nested_dict(temp_d)

d_2021 = {}

h_lst = list(k['industry'])

for i in h_lst:
    d_2021[i] = temp_d[i]

df_industry_2021 = pd.DataFrame.from_dict(d_2021, orient='index')
df_industry_2021.fillna(0, inplace = True)
df_industry_2021 = df_industry_2021.sort_values(by=df_industry_2021.index[0], ascending=False, axis=1)
df_industry_2021

# -----------------------------------------
# Heatmap of job title within industry 2021
# -----------------------------------------

student_2018 = data_2018.loc[data_2018['Q7'] == 'I am a student']
workforce_2018 = data_2018.loc[data_2018['Q7'] != 'I am a student']

professional_2018 = workforce_2018[workforce_2018['Q7'].notna()]
professional_2018['Q6'] = professional_2018['Q6'].map(job_title)

industry_2018 = professional_2018['Q7'].unique()

df_2018 = professional_2018[['Q6','Q7']]

temp_d = {}

for industry in industry_2018:
    temp_df = df_2018[df_2018['Q7'] == industry]
    temp_dict = dict(temp_df['Q6'].value_counts())
    temp_d[industry] = temp_dict

def sorted_simple_dict(d):
    return {k: v for k, v in sorted(d.items())}

def sorted_once_nested_dict(d):
    return {k: sorted_simple_dict(v) for k, v in sorted(d.items())}

temp_d = sorted_once_nested_dict(temp_d)

d_2018 = {}

for i in h_lst:
    d_2018[i] = temp_d[i]

df_industry_2018 = pd.DataFrame.from_dict(d_2018, orient='index')
df_industry_2018.fillna(0, inplace = True)
df_industry_2018 = df_industry_2018.sort_values(by=df_industry_2018.index[0], ascending=False, axis=1)

# --- Plot ---

fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize=(20,10))

ax1 = plt.subplot(121)
plt.title('2018 industry workforce distribution heatmap')
sns.heatmap(df_industry_2018, annot=True, annot_kws = {"size": 8}, linewidth = 0.5, fmt='g', square=True, cmap = 'BuGn')

fig.subplots_adjust(wspace=0.2)

ax2 = plt.subplot(122)
plt.title('2021 industry workforce distribution heatmap')
sns.heatmap(df_industry_2021, annot=True, annot_kws = {"size": 8}, linewidth = 0.5, fmt='g', square=True, cmap = 'BuGn')

plt.axvline(x = 2, ymin= -0.05, ymax= 1.1, color='red', linestyle='solid', linewidth=5)
plt.axvline(x = 3, ymin= -0.05, ymax= 1.1, color='red', linestyle='solid', linewidth=5)

frame1 = plt.gca()
frame1.axes.yaxis.set_ticklabels([])

plt.savefig("9.1 industry workforce heatmap.png", bbox_inches='tight')

plt.show()

In [ ]:
# --- Job title and Tasks during the work ---

from math import pi
# activities at work

# Q24_Part_1	Q24_Part_2	Q24_Part_3	Q24_Part_4	Q24_Part_5	Q24_Part_6	Q24_Part_7	Q24_OTHER

activities_2021 = ['Q24_Part_1','Q24_Part_2','Q24_Part_3','Q24_Part_4','Q24_Part_5','Q24_Part_6','Q24_Part_7','Q24_OTHER']

role_2021 = ['Q5']

role_activities_2021 = ['Q5','Q24_Part_1','Q24_Part_2','Q24_Part_3','Q24_Part_4','Q24_Part_5','Q24_Part_6','Q24_Part_7','Q24_OTHER']

job_name = ['Data Scientist', 'Machine Learning Engineer',  'Software Engineer', 'Data Analyst', 'Research Scientist']

df_role_act_2021 = pros_2021[role_activities_2021]

df_role_act_2021 = df_role_act_2021[df_role_act_2021['Q5'].isin(job_name)]

df_role_act_2021[activities_2021] = df_role_act_2021[activities_2021].notnull().astype('int')

# --------------

df_SE = df_role_act_2021[df_role_act_2021['Q5'] == 'Software Engineer']
df_SE = df_SE.groupby(by='Q5', dropna=False).sum()
df_SE = df_SE.rename(columns = {'Q24_Part_1': 'Analyze data', 
                     'Q24_Part_2': 'Build infrastructure', 
                     'Q24_Part_3': 'Build ML prototypes', 
                     'Q24_Part_4': 'Deploy & Improve',
                     'Q24_Part_5': 'Improve existing ML',
                     'Q24_Part_6': 'Do research',
                     'Q24_Part_7': 'None',
                     'Q24_OTHER': 'Other'
                    })

df_SE = df_SE.T.reset_index()

# --------------

df_DS = df_role_act_2021[df_role_act_2021['Q5'] == 'Data Scientist']
df_DS = df_DS.groupby(by='Q5', dropna=False).sum()
df_DS = df_DS.rename(columns = {'Q24_Part_1': 'Analyze data', 
                     'Q24_Part_2': 'Build infrastructure', 
                     'Q24_Part_3': 'Build ML prototypes', 
                     'Q24_Part_4': 'Deploy & Improve',
                     'Q24_Part_5': 'Improve existing ML',
                     'Q24_Part_6': 'Do research',
                     'Q24_Part_7': 'None',
                     'Q24_OTHER': 'Other'
                    })
df_DS = df_DS.T.reset_index()

# -------------

df_MLE = df_role_act_2021[df_role_act_2021['Q5'] == 'Machine Learning Engineer']

df_MLE = df_MLE.groupby(by='Q5', dropna=False).sum()
df_MLE = df_MLE.rename(columns = {'Q24_Part_1': 'Analyze data', 
                     'Q24_Part_2': 'Build infrastructure', 
                     'Q24_Part_3': 'Build ML prototypes', 
                     'Q24_Part_4': 'Deploy & Improve',
                     'Q24_Part_5': 'Improve existing ML',
                     'Q24_Part_6': 'Do research',
                     'Q24_Part_7': 'None',
                     'Q24_OTHER': 'Other'
                    })
df_MLE = df_MLE.T.reset_index()

#----------

df_MLE['percent'] = (df_MLE['Machine Learning Engineer'] / 
                  df_MLE['Machine Learning Engineer'].sum()) * 100
df_MLE = df_MLE.drop('Machine Learning Engineer', axis = 1)

df_DS['percent'] = (df_DS['Data Scientist'] / 
                  df_DS['Data Scientist'].sum()) * 100
df_DS = df_DS.drop('Data Scientist', axis = 1)

df_SE['percent'] = (df_SE['Software Engineer'] / 
                  df_SE['Software Engineer'].sum()) * 100
df_SE = df_SE.drop('Software Engineer', axis = 1)

# -------

theta = np.arange(len(df_MLE) + 1) / float(len(df_MLE)) * 2 * np.pi
values = df_MLE['percent'].values
values = np.append(values, values[0])

values1 = df_DS['percent'].values
values1 = np.append(values1, values1[0])

values2 = df_SE['percent'].values
values2 = np.append(values2, values2[0])

# --- Plot ---

fig, ax = plt.subplots(figsize=(10,10))
plt.suptitle("Tasks in workplace. MLE vs. DS vs. SE", y = 1)

ax = plt.subplot(projection="polar")

# draw the polygon and the mark the points for each angle/value combination
l1, = ax.plot(theta, values, color="C2", marker="o", label="MLE")
l2, = ax.plot(theta, values1, color="C3", marker="o", label="DS")
l3, = ax.plot(theta, values2, color="C4", marker="o", label="SE")

plt.xticks(theta[:-1], df_MLE['index'], color='grey', size=20)
ax.tick_params(pad=40) # to increase the distance of the labels to the plot
# fill the area of the polygon with green and some transparency
ax.fill(theta, values, 'green', alpha=0.1)
ax.fill(theta, values1, 'red', alpha=0.1)
ax.fill(theta, values2, 'purple', alpha=0.1)
ax.legend(loc = 'lower right',bbox_to_anchor = (1.3,0.1))

plt.savefig("9.2 Tasks in workplace. MLE vs. DS vs. SE.png", bbox_inches='tight')

plt.show()

In [ ]:
test = get_professionals(data_2021, 'Q5')

# ----------------------------------------------------------------------------------------
# 2021 (#18 and #19), 2020 (#18 and #19), 2019 (#26 and #27) About Computer vision and NLP
# ----------------------------------------------------------------------------------------

# ---------------------------------
# COMPUTER VISION YES OR NO in 2021
# ---------------------------------

vision = test[['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5','Q18_Part_6','Q18_OTHER']]
nlp = test[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER']]
vision = vision.fillna(0)
vision[vision != 0] = 1
vision_df = vision[vision==True].count(axis=0).rename_axis('Algo').reset_index(name='counts')
vision_df = vision_df.set_index('Algo').T
stuff_2020 = vision_df

vision['yes'] = vision[['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5','Q18_OTHER']].sum(axis = 1)
vision['yes'] = vision['yes'].apply(lambda x: x>=1)
#vision['yes'].value_counts()
vision['no'] = (vision['yes'].apply(lambda x: x == 0) | vision['Q18_Part_6'].apply(lambda x: x == 1))
#vision['no'].value_counts()

vision['Q20'] = test['Q20']
vision = vision.drop(columns=['Q18_Part_1','Q18_Part_2','Q18_Part_3','Q18_Part_4','Q18_Part_5','Q18_Part_6','Q18_OTHER'])
vision.replace({False: 0, True: 1}, inplace=True)
vision_df = vision[vision['yes'] == 1].groupby('Q20').size()

total_boss = vision['Q20'].value_counts()

boss = pd.DataFrame(vision_df)
total_boss = pd.DataFrame(total_boss)
final_boss = total_boss.join(boss)
final_boss.rename(columns = {final_boss.columns[0]: 'NO', final_boss.columns[1]: 'YES'}, inplace = True)
final_boss = final_boss[::-1]

# --- Plot ---

fig, ax = plt.subplots(figsize=(10,10))

final_boss.plot(kind='barh',
                ax = ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
ax.set(title = "Computer Vision Yes / No",
       xlabel = "the number of respondents",
       ylabel = "Industry")

plt.rc('font', size=12)

for i,j in (zip(ax.containers[0], ax.containers[1])):

    perc = j.get_width() / i.get_width()
    perc = (perc*100).round(1)
    non_perc = (100 - perc).round(1)
    
    width = i.get_width()
    height = i.get_height()
    x, y = i.get_xy()
    ax.annotate(f'{non_perc}%', (x + width, y + height*1.02), ha="left", va="center")
    
    width2 = j.get_width()
    height2 = j.get_height()
    x2, y2 = j.get_xy() 
    ax.annotate(f'{perc}%', (x + width2, y2 + height2*1.02), ha="left", va="center")
    
plt.rcParams['font.size'] = 20

plt.savefig("10.1 Computer Vision.png", bbox_inches='tight')
plt.show()

In [ ]:
# ---------------------
# NLP YES OR NO in 2021
# ---------------------

nlp = test[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER']]
nlp = nlp.fillna(0)
nlp[nlp != 0] = 1
nlp_df = nlp[nlp==True].count(axis=0).rename_axis('Algo').reset_index(name='counts')
nlp_df = nlp_df.set_index('Algo').T

nlp['yes'] = nlp[['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_OTHER']].sum(axis = 1)
nlp['yes'] = nlp['yes'].apply(lambda x: x>=1)
#vision['yes'].value_counts()
nlp['no'] = (nlp['yes'].apply(lambda x: x == 0) | nlp['Q19_Part_5'].apply(lambda x: x == 1))
#vision['no'].value_counts()

nlp['Q20'] = test['Q20']
nlp = nlp.drop(columns=['Q19_Part_1','Q19_Part_2','Q19_Part_3','Q19_Part_4','Q19_Part_5','Q19_OTHER'])
nlp.replace({False: 0, True: 1}, inplace=True)
nlp_df = nlp[nlp['yes'] == 1].groupby('Q20').size()

total_boss2 = nlp['Q20'].value_counts()

boss2 = pd.DataFrame(nlp_df)
total_boss2 = pd.DataFrame(total_boss2)
final_boss2 = total_boss.join(boss2)
final_boss2.rename(columns = {final_boss2.columns[1]: 'hello'}, inplace = True)
#final_boss2['perc'] = final_boss2['hello'] * 100 / final_boss2['Q20']
final_boss2.rename(columns = {final_boss2.columns[0]: 'NO', final_boss2.columns[1]: 'YES'}, inplace = True)
final_boss2 = final_boss2[::-1]

# --------------- 
#  SUBPLOTS - 1x3
# ---------------

fig, ax = plt.subplots(figsize=(10,10))

final_boss2.plot(kind='barh',
                 ax = ax,
                color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
ax.set(title = "NLP Yes / No",
       xlabel = "the number of respondents",
       ylabel = "Industry")

plt.rc('font', size=12)

for i,j in (zip(ax.containers[0], ax.containers[1])):

    perc = j.get_width() / i.get_width()
    perc = (perc*100).round(1)
    non_perc = (100 - perc).round(2)
    
    width = i.get_width()
    height = i.get_height()
    x, y = i.get_xy()
    ax.annotate(f'{non_perc}%', (x + width, y + height*1.02), ha="left", va="center")
    
    width2 = j.get_width()
    height2 = j.get_height()
    x2, y2 = j.get_xy() 
    ax.annotate(f'{perc}%', (x + width2, y2 + height2*1.02), ha="left", va="center")

plt.rcParams['font.size'] = 20

plt.savefig("10.2 NLP.png", bbox_inches='tight')
plt.show()

In [ ]:
ML_framework_2021 = ['Q16_Part_1',
                     'Q16_Part_2',
                     'Q16_Part_3',
                     'Q16_Part_4',
                     'Q16_Part_5',
                     'Q16_Part_6',
                     'Q16_Part_7',
                     'Q16_Part_8',
                     'Q16_Part_9',
                     'Q16_Part_10',
                     'Q16_Part_11',
                     'Q16_Part_12',
                     'Q16_Part_13',
                     'Q16_Part_14',
                     'Q16_Part_15',
                     'Q16_Part_16',
                     'Q16_Part_17'
                     ,'Q16_OTHER']

df_ML_framework_2021 = pros_2021[ML_framework_2021]
count_ML_framework_2021 = pd.Series(df_ML_framework_2021[ML_framework_2021].squeeze().values.ravel()).value_counts()

df_count_ML_framework_2021 = pd.DataFrame(count_ML_framework_2021)
df_count_ML_framework_2021 = df_count_ML_framework_2021.reset_index()
df_count_ML_framework_2021.columns = ['ML Frameworks', 'Counts']

df_count_ML_framework_2021 = df_count_ML_framework_2021.set_index('ML Frameworks').T
df_count_ML_framework_2021.columns = df_count_ML_framework_2021.columns.str.strip()

ML_framework_2020 = ['Q16_Part_1',
                     'Q16_Part_2',
                     'Q16_Part_3',
                     'Q16_Part_4',
                     'Q16_Part_5',
                     'Q16_Part_6',
                     'Q16_Part_7',
                     'Q16_Part_8',
                     'Q16_Part_9',
                     'Q16_Part_10',
                     'Q16_Part_11',
                     'Q16_Part_12',
                     'Q16_Part_13',
                     'Q16_Part_14',
                     'Q16_Part_15',
                     'Q16_OTHER']

df_ML_framework_2020 = pros_2020[ML_framework_2020]
count_ML_framework_2020 = pd.Series(df_ML_framework_2020[ML_framework_2020].squeeze().values.ravel()).value_counts()

df_count_ML_framework_2020 = pd.DataFrame(count_ML_framework_2020)
df_count_ML_framework_2020 = df_count_ML_framework_2020.reset_index()
df_count_ML_framework_2020.columns = ['ML Frameworks', 'Counts']

df_count_ML_framework_2020 = df_count_ML_framework_2020.set_index('ML Frameworks').T
df_count_ML_framework_2020.columns = df_count_ML_framework_2020.columns.str.strip()

ML_framework_2019 = ['Q28_Part_1',
                     'Q28_Part_2',
                     'Q28_Part_3',
                     'Q28_Part_4',
                     'Q28_Part_5',
                     'Q28_Part_6',
                     'Q28_Part_7',
                     'Q28_Part_8',
                     'Q28_Part_9',
                     'Q28_Part_10',
                     'Q28_Part_11',
                     'Q28_Part_12',
                     'Q28_OTHER_TEXT']

df_ML_framework_2019 = pros_2019[ML_framework_2019]
count_ML_framework_2019 = pd.Series(df_ML_framework_2019[ML_framework_2019].squeeze().values.ravel()).value_counts()

df_count_ML_framework_2019 = pd.DataFrame(count_ML_framework_2019)
df_count_ML_framework_2019 = df_count_ML_framework_2019.reset_index()
df_count_ML_framework_2019.columns = ['ML Frameworks', 'Counts']

# Some clean up
df_count_ML_framework_2019 = df_count_ML_framework_2019.set_index('ML Frameworks').T
df_count_ML_framework_2019.columns = df_count_ML_framework_2019.columns.str.strip()
df_count_ML_framework_2019 = df_count_ML_framework_2019[['Scikit-learn', 'Keras', 'TensorFlow', 'RandomForest', 'Xgboost', 'PyTorch', 'LightGBM', 'None', 'Caret', 'Spark MLib',
                           'Fast.ai', 'Other']]

merged_ML_framcework = pd.concat([df_count_ML_framework_2019, df_count_ML_framework_2020, df_count_ML_framework_2021])
merged_ML_framcework.index = ['2019','2020','2021']
merged_ML_framcework = merged_ML_framcework.fillna(0)
merged_ML_framcework['total'] = merged_ML_framcework.sum(axis=1)
merged_ML_framcework[['TensorFlow', 'PyTorch', 'Keras', 'total']]

merged_ML_framcework['TensorFlow'] = merged_ML_framcework['TensorFlow']/merged_ML_framcework['total']
merged_ML_framcework['PyTorch'] = merged_ML_framcework['PyTorch']/merged_ML_framcework['total']
merged_ML_framcework['Keras'] = merged_ML_framcework['Keras']/merged_ML_framcework['total']

fig, ax = plt.subplots(figsize = (10,10))

merged_ML_framcework[['TensorFlow', 'PyTorch','Keras']].plot(kind='bar',
                                                     ax=ax,
                                                     stacked = True,
                                                     color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])

ax.set_title("ML frameworks usage proportion", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Percentage', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.4, 0.6), fontsize=20)

plt.savefig("11.1 Softpower ML frameworks.png", bbox_inches='tight')
plt.show()

In [ ]:
hosting_notebook_2021 = ['Q10_Part_1',
                    'Q10_Part_2',
                    'Q10_Part_3',
                    'Q10_Part_4',
                    'Q10_Part_5',
                    'Q10_Part_6',
                    'Q10_Part_7',
                    'Q10_Part_8',
                    'Q10_Part_9',
                    'Q10_Part_10',
                    'Q10_Part_11',
                    'Q10_Part_12',
                    'Q10_Part_13',
                    'Q10_Part_14',
                    'Q10_Part_15',
                    'Q10_Part_16',
                    'Q10_OTHER'
                   ]

df_hosting_notebook_2021 = pros_2021[hosting_notebook_2021]
count_hosting_notebook_2021 = pd.Series(df_hosting_notebook_2021[hosting_notebook_2021].squeeze().values.ravel()).value_counts()

df_count_hosting_notebook_2021 = pd.DataFrame(count_hosting_notebook_2021)
df_count_hosting_notebook_2021 = df_count_hosting_notebook_2021.reset_index()
df_count_hosting_notebook_2021.columns = ['ML Frameworks', 'Counts']

df_count_hosting_notebook_2021 = df_count_hosting_notebook_2021.set_index('ML Frameworks').T
df_count_hosting_notebook_2021.columns = df_count_hosting_notebook_2021.columns.str.strip()

hosting_notebook_2020 = ['Q10_Part_1',
                         'Q10_Part_2',
                         'Q10_Part_3',
                         'Q10_Part_4',
                         'Q10_Part_5',
                         'Q10_Part_6',
                         'Q10_Part_7',
                         'Q10_Part_8',
                         'Q10_Part_9',
                         'Q10_Part_10',
                         'Q10_Part_11',
                         'Q10_Part_12',
                         'Q10_Part_13',
                         'Q10_OTHER']

df_hosting_notebook_2020 = pros_2020[hosting_notebook_2020]
count_hosting_notebook_2020 = pd.Series(df_hosting_notebook_2020[hosting_notebook_2020].squeeze().values.ravel()).value_counts()

df_count_hosting_notebook_2020 = pd.DataFrame(count_hosting_notebook_2020)
df_count_hosting_notebook_2020 = df_count_hosting_notebook_2020.reset_index()
df_count_hosting_notebook_2020.columns = ['ML Frameworks', 'Counts']

df_count_hosting_notebook_2020 = df_count_hosting_notebook_2020.set_index('ML Frameworks').T
df_count_hosting_notebook_2020.columns = df_count_hosting_notebook_2020.columns.str.strip()

hosting_notebook_2019 =['Q17_Part_1',
                        'Q17_Part_2',
                        'Q17_Part_3',
                        'Q17_Part_4',
                        'Q17_Part_5',
                        'Q17_Part_6',
                        'Q17_Part_7',
                        'Q17_Part_8',
                        'Q17_Part_9',
                        'Q17_Part_10',
                        'Q17_Part_11',
                        'Q17_Part_12',
                        'Q17_OTHER_TEXT']

df_hosting_notebook_2019 = pros_2019[hosting_notebook_2019]
count_hosting_notebook_2019 = pd.Series(df_hosting_notebook_2019[hosting_notebook_2019].squeeze().values.ravel()).value_counts()

df_count_hosting_notebook_2019 = pd.DataFrame(count_hosting_notebook_2019)
df_count_hosting_notebook_2019 = df_count_hosting_notebook_2019.reset_index()
df_count_hosting_notebook_2019.columns = ['ML Frameworks', 'Counts']

df_count_hosting_notebook_2019 = df_count_hosting_notebook_2019.set_index('ML Frameworks').T
df_count_hosting_notebook_2019.columns = df_count_hosting_notebook_2019.columns.str.strip()

# --- Clean up for merge ---

df_count_hosting_notebook_2019 = df_count_hosting_notebook_2019[['Kaggle Notebooks (Kernels)', 'Google Colab', 'Binder / JupyterHub', 'Microsoft Azure Notebooks',
                               'AWS Notebook Products (EMR Notebooks, Sagemaker Notebooks, etc)',
                               'IBM Watson Studio',
                               'Paperspace / Gradient',
                               'FloydHub',
                              'Code Ocean']]

df_count_hosting_notebook_2019['total'] = df_count_hosting_notebook_2019.sum(axis=1)
df_count_hosting_notebook_2019[['Kaggle Notebooks (Kernels)', 
                                'Google Colab', 
                                'Binder / JupyterHub', 
                                'AWS Notebook Products (EMR Notebooks, Sagemaker Notebooks, etc)',
                                'total']]

df_count_hosting_notebook_2019 = df_count_hosting_notebook_2019.rename(columns = 
                                     {'Kaggle Notebooks (Kernels)': 'Kaggle Notebooks', 
                                'Google Colab': 'Colab Notebooks',
                                      'Microsoft Azure Notebooks': 'Azure Notebooks',
                                'Binder / JupyterHub': 'Binder / JupyterHub', 
                                'AWS Notebook Products (EMR Notebooks, Sagemaker Notebooks, etc)': 'Amazon Sagemaker Studio',
                                'total': 'total'
                                     })

df_count_hosting_notebook_2020 = df_count_hosting_notebook_2020.drop('None', axis = 1)
df_count_hosting_notebook_2020['total'] = df_count_hosting_notebook_2020.sum(axis = 1)
df_count_hosting_notebook_2021 = df_count_hosting_notebook_2021.drop('None', axis = 1)
df_count_hosting_notebook_2021['total'] = df_count_hosting_notebook_2021.sum(axis = 1)

df_count_hosting_notebook_2021 = df_count_hosting_notebook_2021.rename(columns = {"Amazon Sagemaker Studio Notebooks": "Amazon Sagemaker Studio"})



#  --- Now merge ---

hosting_notebook_merge = pd.concat([df_count_hosting_notebook_2019, df_count_hosting_notebook_2020, df_count_hosting_notebook_2021])
hosting_notebook_merge.index = ['2019', '2020', '2021']
hosting_notebook_merge = hosting_notebook_merge.fillna(0)
top_notebooks_used = hosting_notebook_merge[['Kaggle Notebooks', 
                        'Colab Notebooks', 
                        'Binder / JupyterHub', 
                        'Azure Notebooks', 
                        'IBM Watson Studio', 
                        'Amazon Sagemaker Studio', 
                        'total']]

top_notebooks_used_perc = top_notebooks_used[['Kaggle Notebooks', 
                        'Colab Notebooks', 
                        'Binder / JupyterHub', 
                        'Azure Notebooks', 
                        'IBM Watson Studio', 
                        'Amazon Sagemaker Studio', 
                        'total']].div(top_notebooks_used['total'], axis = 0)

# --- Plot ---

fig, ax = plt.subplots(figsize = (10,10))

top_notebooks_used_perc[['Kaggle Notebooks', 
                        'Colab Notebooks', 
                        'Binder / JupyterHub', 
                        'Azure Notebooks', 
                        'IBM Watson Studio', 
                        'Amazon Sagemaker Studio']].plot(kind='bar', 
                                                         stacked = True,
                                                        ax=ax,
                                                     color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])


ax.set_title("Proportion of hosted notebook products usage", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Percentage', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.7, 0.6), fontsize=20)

plt.savefig("11.2 Softpower Notebook.png", bbox_inches='tight')
plt.show()

In [ ]:
share_deploy_2021 = [
    'Q39_Part_1',
    'Q39_Part_2',
    'Q39_Part_3',
    'Q39_Part_4',
    'Q39_Part_5',
    'Q39_Part_6',
    'Q39_Part_7',
    'Q39_Part_8',
    'Q39_Part_9',
    'Q39_OTHER'
]

df_share_deploy_2021 = pros_2021[share_deploy_2021]
count_share_deploy_2021 = pd.Series(df_share_deploy_2021[share_deploy_2021].squeeze().values.ravel()).value_counts()

df_count_share_deploy_2021 = pd.DataFrame(count_share_deploy_2021)
df_count_share_deploy_2021 = df_count_share_deploy_2021.reset_index()
df_count_share_deploy_2021.columns = ['Platforms', 'Counts']

df_count_share_deploy_2021 = df_count_share_deploy_2021.set_index('Platforms').T
df_count_share_deploy_2021.columns = df_count_share_deploy_2021.columns.str.strip()

share_deploy_2020 =[
    'Q36_Part_1',
    'Q36_Part_2',
    'Q36_Part_3',
    'Q36_Part_4',
    'Q36_Part_5',
    'Q36_Part_6',
    'Q36_Part_7',
    'Q36_Part_8',
    'Q36_Part_9',
    'Q36_OTHER']

df_share_deploy_2020 = pros_2020[share_deploy_2020]
count_share_deploy_2020 = pd.Series(df_share_deploy_2020[share_deploy_2020].squeeze().values.ravel()).value_counts()

df_count_share_deploy_2020 = pd.DataFrame(count_share_deploy_2020)
df_count_share_deploy_2020 = df_count_share_deploy_2020.reset_index()
df_count_share_deploy_2020.columns = ['Platforms', 'Counts']

df_count_share_deploy_2020 = df_count_share_deploy_2020.set_index('Platforms').T
df_count_share_deploy_2020.columns = df_count_share_deploy_2020.columns.str.strip()

# --- Merge ---
merged_share_deploy = pd.concat([df_count_share_deploy_2020, df_count_share_deploy_2021])
merged_share_deploy.index = ['2020', '2021']

merged_share_deploy = merged_share_deploy.drop('I do not share my work publicly', axis = 1)
merged_share_deploy['total'] = merged_share_deploy.sum(axis = 1)
merged_share_deploy = merged_share_deploy.div(merged_share_deploy['total'], axis = 0)
merged_share_deploy = merged_share_deploy.drop('total', axis = 1)

fig, ax = plt.subplots(figsize = (10,10))

merged_share_deploy[['GitHub', 'Kaggle', 'Personal blog', 'Shiny']].plot(kind='bar', 
                                                         stacked = True,
                                                        ax=ax,
                                                     color = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])


ax.set_title("Platform to share or deploy ML applications", fontsize ='large', pad=20)
ax.set_xlabel('Year', fontsize='medium', labelpad=20)
ax.set_ylabel('Percentage', fontsize='medium', labelpad=20)
ax.legend(bbox_to_anchor = (1.45, 0.6), fontsize=20)

plt.savefig("11.3 Softpower ML repository.png", bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib as mpl
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS

#mpl.rcParams['figure.figsize']=(8.0,6.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=100             #72 
mpl.rcParams['figure.subplot.bottom']=.1 


stopwords = set(STOPWORDS)
f = open("../input/big-tech-keynotes/2020_keynote_amzn.txt", "r", encoding='utf-8')
file_content = f.read()

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=40, 
                          random_state=42
                         ).generate(file_content)

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("amzn_keynote_clouds.png", dpi=900)

In [ ]:
stopwords = set(STOPWORDS)
f = open("../input/big-tech-keynotes/2021_google_io.txt", "r", encoding='utf-8')
file_content = f.read()

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=40, 
                          random_state=42
                         ).generate(file_content)

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("google_io_2021.png", dpi=900)

In [ ]:
stopwords = set(STOPWORDS)
f = open("../input/big-tech-keynotes/2021_ignite_2021.txt", "r", encoding='utf-8')
file_content = f.read()

wordcloud = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=200,
                          max_font_size=40, 
                          random_state=42
                         ).generate(file_content)

print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("msft_ignite_2021.png", dpi=900)